# Survival Analysis of VC-Backed Startups

---

**Author:** Daniele Parini 
**Date:** 2025  
**Institution:** HEC-Lausanne

---

### Abstract

This notebook implements a comprehensive survival analysis framework to predict the probability and timing of exit events (IPO or acquisition) for VC-backed startups. We employ a competing risks methodology where IPO and M&A are treated as mutually exclusive terminal events.

### Methodology Overview

1. **Data Preparation**: Crunchbase and Jay Ritter datasets
2. **Exploratory Analysis**: Kaplan-Meier curves, cumulative incidence functions
3. **Statistical Models**: Cox Proportional Hazards with stratification
4. **Parametric Models**: Weibull, Log-Normal, Log-Logistic AFT
5. **Machine Learning**: Random Survival Forest, Gradient Boosting, DeepSurv
6. **Model Evaluation**: C-index, Brier Score, calibration plots

---
### 1. Environment Setup & Library Imports

In [ ]:
%load_ext autoreload
# Autoreload everything
%autoreload 2

import config

In [ ]:
# Core libraries
import warnings
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd

import re

warnings.filterwarnings("ignore")

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Survival analysis
from matplotlib.ticker import PercentFormatter

# Statistical testing
from scipy import stats

# ML preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Plot settings
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 12
sns.set_style("whitegrid")
sns.set_palette("husl")

# Random seed
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 2. Data Loading & Initial Inspection

### Objectives of this section:
1. **Load** the Crunchbase and Jay Ritter datasets
2. **Explore** the structure, quality, and distributions of the data
3. **Analyze** funding, success rates, and correlations
4. **Merge** the datasets using a conservative strategy to validate IPO events
5. **Identify** key features for survival analysis

### Datasets used:
| Dataset | Description | Primary use |
|---------|-------------|----------------|
| Crunchbase | ~196K companies (1970-2013) | Main dataset for survival |
| Jay Ritter | ~15,700 IPO USA (1975-2025) | IPO validation + market variables |

### 2.1 Setup and Path Definition

In [ ]:
import os
import shutil

import kagglehub


config.DATA_DIR.mkdir(parents=True, exist_ok=True)

print(f"Project Root: {config.PROJECT_ROOT}")
print(f"Data Directory: {config.DATA_DIR}")

config.RAW_DIR.mkdir(exist_ok=True)

# Download latest version
path = kagglehub.dataset_download("justinas/startup-investments")
print("Path to dataset files:", path)
print(f"\nCopying files to: {config.RAW_DIR}")

for item in os.listdir(path):
    source_path = os.path.join(path, item)
    dest_path = os.path.join(config.RAW_DIR, item)

    if os.path.isfile(source_path):
        shutil.copy2(source_path, dest_path)
        print(f"  Copied: {item}")
    elif os.path.isdir(source_path):
        # If it's a directory, copy entire directory
        shutil.copytree(source_path, dest_path, dirs_exist_ok=True)
        print(f"  Copied directory: {item}")

print(f"\n Dataset files saved in: {config.RAW_DIR}")

In [ ]:
# =============================================================================
# 2.1 SETUP E DEFINIZIONE PATHS
# =============================================================================

warnings.filterwarnings("ignore")

# Plot settings
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["font.size"] = 11
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12
sns.set_style("whitegrid")
sns.set_palette("husl")

# -----------------------------------------------------------------------------
# Paths configuration
# -----------------------------------------------------------------------------
config.RAW_DIR = config.DATA_DIR / "raw"

print("ENVIRONMENT CONFIGURATION")

print(f" Raw data path: {config.RAW_DIR}")

# Verifica esistenza directory
if os.path.exists(config.RAW_DIR):
    files = [f for f in os.listdir(config.RAW_DIR) if f.endswith((".csv", ".xlsx"))]
    print(f"\nDirectory trovata con {len(files)} file dati:")
    for f in sorted(files):
        size_mb = os.path.getsize(os.path.join(config.RAW_DIR, f)) / (1024**2)
        print(f" {f} ({size_mb:.2f} MB)")
else:
    print(f"\n ERROR: Directory not found: {config.RAW_DIR}")
    print("First, run the kagglehub download script")

In [ ]:
# =============================================================================
# 2.2 CARICAMENTO DATASET CRUNCHBASE
# =============================================================================


def load_csv_safe(filepath, parse_dates=None, **kwargs):
    """
    Load a CSV file with error handling and logging.
    
    Parameters
    ----------
    filepath : str
        Full path to the CSV file
    parse_dates : list of str, optional
        Column names to parse as datetime objects
    **kwargs : dict
        Additional keyword arguments passed to pd.read_csv
        
    Returns
    -------
    pd.DataFrame or None
        Loaded dataframe if successful, None if error occurs
    """
    try:
        df = pd.read_csv(filepath, parse_dates=parse_dates, low_memory=False, **kwargs)
        filename = os.path.basename(filepath)
        print(f" {filename}: {df.shape[0]:,} righe x {df.shape[1]} colonne")
        return df
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return None
    except Exception as e:
        print(f"Error loading {filepath}: {str(e)[:100]}")
        return None

# -----------------------------------------------------------------------------
# Crunchbase file upload
# -----------------------------------------------------------------------------
print("=" * 70)
print("LOADING CRUNCHBASE DATASET")
print("=" * 70)

crunchbase = {}

# Critical files for survival analysis
critical_files = {
    "objects": {
        "parse_dates": [
            "founded_at",
            "closed_at",
            "first_funding_at",
            "last_funding_at",
            "first_milestone_at",
        ]
    },
    "funding_rounds": {"parse_dates": ["funded_at", "created_at"]},
    "acquisitions": {"parse_dates": ["acquired_at", "created_at"]},
    "ipos": {"parse_dates": ["public_at", "created_at"]},
}

# Supplementary files for enrichment
supplementary_files = {
    "offices": {},
    "investments": {},
    "people": {},
    "relationships": {},
    "degrees": {},
    "funds": {"parse_dates": ["funded_at"]},
    "milestones": {"parse_dates": ["milestone_at"]},
}

print("\n--- Critical files ---")
for name, params in critical_files.items():
    filepath = os.path.join(config.RAW_DIR, f"{name}.csv")
    crunchbase[name] = load_csv_safe(filepath, **params)

print("\n--- Supplementary files ---")
for name, params in supplementary_files.items():
    filepath = os.path.join(config.RAW_DIR, f"{name}.csv")
    crunchbase[name] = load_csv_safe(filepath, **params)

# Summary statistics
loaded_count = sum(1 for v in crunchbase.values() if v is not None)
print(f"\n File loaded: {loaded_count}/{len(crunchbase)}")

In [ ]:
# =============================================================================
# 2.3 CARICAMENTO DATASET JAY RITTER
# =============================================================================

import requests

# 1. Check if file exists; if not, download it
if not config.RITTER_PATH.exists():
    print(f"File non trovato: {config.RITTER_PATH}")
    print(f"Tentativo di download da: {config.RITTER_URL} ...")

    try:
        # Ensure the directory exists
        config.RITTER_PATH.parent.mkdir(parents=True, exist_ok=True)

        # Download the file
        response = requests.get(config.RITTER_URL, timeout=30)
        response.raise_for_status()  # Raises error if status code is not 200 (OK)

        with open(config.RITTER_PATH, "wb") as f:
            f.write(response.content)

        print("Download completed.")

    except Exception as e:
        print(f" Errore critico durante il download: {e}")
        # We don't stop here; we let the next block handle the missing file error gracefully

# 2. Attempt to load the data
try:
    ritter = pd.read_excel(config.RITTER_PATH)
    print(f" IPO-age.xlsx: {ritter.shape[0]:,} righe × {ritter.shape[1]} colonne")
    print(f"\n Colonne originali:")
    print(f"   {ritter.columns.tolist()}")

except FileNotFoundError:
    # This block runs if the download failed or was skipped
    print(f" File non trovato e download fallito: {config.RITTER_PATH}")
    ritter = None

except Exception as e:
    print(f" Errore caricamento: {e}")
    ritter = None

### 2.4 Dataset Exploration: Crunchbase Objects

The `objects.csv` file is the **central table** of the Crunchbase database. It contains all entities:
- Companies (companies/startups)
- People (founders, executives)
- Financial Organizations (VC, PE, angels)

**Focus:** We will filter only `Companies` for survival analysis.

In [ ]:
# =============================================================================
# 2.4 ESPLORATION CRUNCHBASE OBJECTS
# =============================================================================

if crunchbase.get("objects") is not None:
    objects = crunchbase["objects"]

    print("CRUNCHBASE OBJECTS - STRUCTURE")

    # General Info
    print(f"\n Shape: {objects.shape}")
    print(f"\n Info DataFrame:")
    objects.info(verbose=True, show_counts=True)

else:
    print("objects.csv not loaded!")
    objects = None

# Distribuzione Entity Types
if objects is not None:

    print("DISTRIBUTION ENTITY TYPES")

    entity_dist = objects["entity_type"].value_counts()
    print(entity_dist)

# Filtra solo Companies
if objects is not None:
    companies = objects[objects["entity_type"] == "Company"].copy()

    
    print(f"COMPANIES SUBSET: {len(companies):,} companies")
    

    # Status distribution
    print("\n Distribution Status:")
    status_dist = companies["status"].value_counts()
    print(status_dist)

    # Calcola percentuali per survival
    total = len(companies)
    operating = status_dist.get("operating", 0)
    acquired = status_dist.get("acquired", 0)
    ipo = status_dist.get("ipo", 0)
    closed = status_dist.get("closed", 0)

    print(f"\n Interpretation for Survival Analysis:")
    print(f"Operating (censored): {operating:,} ({100 * operating / total:.1f}%)")
    print(f"Acquired (event M&A): {acquired:,} ({100 * acquired / total:.1f}%)")
    print(f"IPO (event IPO): {ipo:,} ({100 * ipo / total:.1f}%)")
    print(f"Closed (censored/failure): {closed:,} ({100 * closed / total:.1f}%)")

# =============================================================================
# ANALYSIS OF ZERO VALUES - CRUNCHBASE COMPANIES
# =============================================================================

if objects is not None:
    
    print("ANALYSIS OF ZERO VALUES - COMPANIES")
    

    # Calculate zero percentages for relevant columns
    key_columns = [
        "name",
        "category_code",
        "status",
        "founded_at",
        "closed_at",
        "country_code",
        "state_code",
        "city",
        "funding_total_usd",
        "funding_rounds",
        "first_funding_at",
        "last_funding_at",
    ]

    null_analysis = []
    for col in key_columns:
        if col in companies.columns:
            null_count = companies[col].isnull().sum()
            null_pct = 100 * null_count / len(companies)
            null_analysis.append(
                {
                    "Column": col,
                    "Null Count": null_count,
                    "Null %": null_pct,
                    "Non-Null": len(companies) - null_count,
                }
            )

    null_df = pd.DataFrame(null_analysis).sort_values("Null %", ascending=False)
    print(null_df.to_string(index=False))

    # Heatmap valori nulli
    fig, ax = plt.subplots(figsize=(14, 6))

    # Seleziona colonne per heatmap
    heatmap_cols = [col for col in key_columns if col in companies.columns]
    null_matrix = companies[heatmap_cols].isnull().astype(int)

    # Sample per visualizzazione (troppi dati per heatmap completa)
    sample_size = min(500, len(null_matrix))
    null_sample = null_matrix.sample(sample_size, random_state=42)

    sns.heatmap(
        null_sample.T,
        cmap="YlOrRd",
        cbar_kws={"label": "Missing (1=Null)"},
        yticklabels=True,
        xticklabels=False,
        ax=ax,
    )
    ax.set_title(
        f"Pattern Null Values - Sample of {sample_size} Companies", fontsize=14
    )
    ax.set_xlabel("Observations (sample)")
    ax.set_ylabel("Variables")

    plt.tight_layout()
    plt.show()

# =============================================================================
# DISTRIBUTION BY CATEGORY AND GEOGRAPHY
# =============================================================================

if objects is not None:
    fig, axes = plt.subplots(1, 2, figsize=(20, 10))

    # Categories
    ax1 = axes[0]
    top_categories = companies["category_code"].value_counts().head(42)
    top_categories.plot(kind="barh", ax=ax1, color="steelblue")
    ax1.set_title(" Categories (Industries)", fontsize=10)
    ax1.set_xlabel("Number of Companies")
    ax1.invert_yaxis()

    # Countries
    ax2 = axes[1]
    top_countries = companies["country_code"].value_counts().head(40)
    top_countries.plot(kind="barh", ax=ax2, color="coral")
    ax2.set_title(" Country", fontsize=10)
    ax2.set_xlabel("Number of Companies")
    ax2.invert_yaxis()

    plt.tight_layout()
    plt.show()

    # Statistiche
    print(f"\n Unique Categories: {companies['category_code'].nunique()}")
    print(f" Unique Country: {companies['country_code'].nunique()}")

# =============================================================================
# DISTRIBUZIONE TEMPORALE - FOUNDING YEARS
# =============================================================================

if objects is not None:
    # Estrai anno di fondazione
    companies["founded_year"] = pd.to_datetime(
        companies["founded_at"], errors="coerce"
    ).dt.year

    # Filtra anni validi
    valid_years = companies[
        (companies["founded_year"] >= 1950) & (companies["founded_year"] <= 2015)
    ]

    fig, axes = plt.subplots(1, 2, figsize=(16, 5))

    # Distribuzione founding years
    ax1 = axes[0]
    valid_years["founded_year"].hist(
        bins=50, ax=ax1, color="steelblue", edgecolor="white", alpha=0.7
    )
    ax1.set_title("Distribution Year of Foundation", fontsize=14)
    ax1.set_xlabel("Year")
    ax1.set_ylabel("Number of Companies")
    ax1.axvline(
        valid_years["founded_year"].median(),
        color="red",
        linestyle="--",
        label=f"Median: {valid_years['founded_year'].median():.0f}",
    )
    ax1.legend()

    # Founding by decade
    ax2 = axes[1]
    valid_years["decade"] = (valid_years["founded_year"] // 10) * 10
    decade_counts = valid_years["decade"].value_counts().sort_index()
    decade_counts.plot(kind="bar", ax=ax2, color="coral", edgecolor="black")
    ax2.set_title("Companies per Decade", fontsize=14)
    ax2.set_xlabel("Decade")
    ax2.set_ylabel("Numbers of Companies")
    ax2.set_xticklabels([f"{int(d)}s" for d in decade_counts.index], rotation=45)

    plt.tight_layout()
    plt.show()

    print(f"\n Range of years of foundation: {valid_years['founded_year'].min():.0f} - {valid_years['founded_year'].max():.0f}")
    print(f" Median: {valid_years['founded_year'].median():.0f}")
    print(f" Companies with founded_year valid: {len(valid_years):,} ({100 * len(valid_years) / len(companies):.1f}%)")

# =============================================================================
# 2.6 ESPLORAZIONE EXIT EVENTS - CRUNCHBASE
# =============================================================================

print("CRUNCHBASE EXIT EVENTS")

# IPOs in Crunchbase
if crunchbase.get("ipos") is not None:
    ipos_cb = crunchbase["ipos"]

    print(f"\n IPO (Crunchbase): {len(ipos_cb):,} record")
    print(f"\n Columns: {ipos_cb.columns.tolist()}")

    # Info
    ipos_cb.info()

    # Sample
    print("Sample IPO records:")

    # Temporal distribution
    ipos_cb["ipo_year"] = pd.to_datetime(ipos_cb["public_at"], errors="coerce").dt.year

    fig, ax = plt.subplots(figsize=(10, 4))
    ipos_cb["ipo_year"].value_counts().sort_index().plot(
        ax=ax, color="green"
    )
    ax.set_title("IPO per Year (Crunchbase)", fontsize=12)
    ax.set_xlabel("Year")
    ax.set_ylabel("Number IPO")
    plt.tight_layout()
    plt.show()

    print(f"\n Range IPO: {ipos_cb['ipo_year'].min():.0f} - {ipos_cb['ipo_year'].max():.0f}")

# Acquisitions in Crunchbase
if crunchbase.get("acquisitions") is not None:
    acquisitions = crunchbase["acquisitions"]

    print(f" ACQUISITIONS (Crunchbase): {len(acquisitions):,} record")


    print(f"\n Columns: {acquisitions.columns.tolist()}")

    # Sample
    print("Sample acquisition records:")
    print(acquisitions.head().to_string())

    # Temporal distribution
    acquisitions["acq_year"] = pd.to_datetime(
        acquisitions["acquired_at"], errors="coerce"
    ).dt.year

    fig, ax = plt.subplots(figsize=(10, 4))
    acquisitions["acq_year"].value_counts().sort_index().plot(
        ax=ax, color="red"
    )
    ax.set_title("Acquisition per Year (Crunchbase)", fontsize=12)
    ax.set_xlabel("Year")
    ax.set_ylabel("Number of Acquisitions")
    plt.tight_layout()
    plt.show()

    # Price statistics
    valid_prices = acquisitions["price_amount"][acquisitions["price_amount"] > 0]
    print(f"\n Acquisition Price Statistics:")
    print(
        f"   Records with price: {len(valid_prices):,} ({100 * len(valid_prices) / len(acquisitions):.1f}%)"
    )
    print(f"   Mean: ${valid_prices.mean() / 1e6:.1f}M")
    print(f"   Median: ${valid_prices.median() / 1e6:.1f}M")
    print(f"   Max: ${valid_prices.max() / 1e9:.2f}B")

In [ ]:
# =============================================================================
# LOADING DATASETS
# =============================================================================

crunchbase = {}

# ACQUISITIONS
filepath = os.path.join(config.RAW_DIR, "acquisitions.csv")
acquisitions = load_csv_safe(filepath, parse_dates=["acquired_at"])
if acquisitions is not None:
    # Drop columns
    acquisitions = acquisitions.drop(columns=[
        'id', 'acquisition_id', 'created_at', 'updated_at', 
        'source_url', 'source_description'
    ], errors='ignore')
    acquisitions = acquisitions.drop_duplicates()
    acquisitions['term_code'] = acquisitions['term_code'].astype('category')
    acquisitions['price_currency_code'] = acquisitions['price_currency_code'].astype('category')

# FUNDING_ROUNDS
filepath = os.path.join(config.RAW_DIR, "funding_rounds.csv")
funding_rounds = load_csv_safe(filepath, parse_dates=["funded_at"])
if funding_rounds is not None:
    # Drop columns
    funding_rounds = funding_rounds.drop(columns=[
        'id', 'created_at', 'updated_at', 'created_by', 'raised_amount', 
        'raised_currency_code', 'pre_money_valuation', 'post_money_valuation_usd',
        'pre_money_currency_code', 'post_money_valuation', 'pre_money_valuation_usd', 
        'post_money_currency_code', 'source_url'
    ], errors='ignore')
    funding_rounds = funding_rounds.drop_duplicates()
    
    # Type conversions
    for col in ['funding_round_type', 'funding_round_code', 'is_first_round', 'is_last_round']:
        if col in funding_rounds.columns:
            funding_rounds[col] = funding_rounds[col].astype('category')
    
    # Fix misclassifications
    funding_rounds.loc[
        (funding_rounds['funding_round_code'] == 'angel') & 
        (funding_rounds['funding_round_type'] == 'series-a'), 
        'funding_round_type'
    ] = 'angel'
    
    funding_rounds.loc[
        (funding_rounds['funding_round_code'] == 'seed') & 
        (funding_rounds['funding_round_type'] == 'series-a'), 
        'funding_round_type'
    ] = 'angel'

# FUNDS
filepath = os.path.join(config.RAW_DIR, "funds.csv")
funds = load_csv_safe(filepath, parse_dates=["funded_at"])
if funds is not None:
    funds = funds.drop(columns=['id', 'created_at', 'updated_at', 'source_url', 'source_description'], errors='ignore')
    funds = funds.drop_duplicates()
    funds['raised_currency_code'] = funds['raised_currency_code'].astype('category')

# INVESTMENTS
filepath = os.path.join(config.RAW_DIR, "investments.csv")
investments = load_csv_safe(filepath)
if investments is not None:
    investments = investments.drop(columns=['id', 'created_at', 'updated_at'], errors='ignore')
    investments = investments.drop_duplicates()

# IPOS
filepath = os.path.join(config.RAW_DIR, "ipos.csv")
ipos = load_csv_safe(filepath, parse_dates=["public_at"])
if ipos is not None:
    ipos = ipos.drop(columns=['id', 'created_at', 'updated_at', 'stock_symbol', 'source_url', 'source_description'], errors='ignore')
    ipos = ipos.drop_duplicates()
    ipos['valuation_currency_code'] = ipos['valuation_currency_code'].astype('category')
    ipos['raised_currency_code'] = ipos['raised_currency_code'].astype('category')

# MILESTONES
filepath = os.path.join(config.RAW_DIR, "milestones.csv")
milestones = load_csv_safe(filepath, parse_dates=["milestone_at"])
if milestones is not None:
    milestones = milestones.drop(columns=['id', 'created_at', 'updated_at', 'source_url', 'milestone_code'], errors='ignore')
    milestones = milestones.drop_duplicates()

# OBJECTS
filepath = os.path.join(config.RAW_DIR, "objects.csv")
objects = load_csv_safe(
    filepath, 
    parse_dates=['founded_at', 'closed_at', 'first_funding_at', 'last_funding_at',
                 'first_milestone_at', 'last_milestone_at']
)
if objects is not None:
    objects = objects.drop(columns=[
        'normalized_name', 'permalink', 'created_at', 'updated_at',
        'first_investment_at', 'last_investment_at', 'created_by',
        'domain', 'twitter_username', 'logo_url', 'overview'
    ], errors='ignore')
    objects = objects.drop_duplicates()
    
    # Type conversions
    for col in ['country_code', 'state_code', 'entity_type', 'category_code', 'status']:
        if col in objects.columns:
            objects[col] = objects[col].astype('category')
    
    for col in ['investment_rounds', 'invested_companies', 'funding_rounds', 
                'funding_total_usd', 'milestones', 'relationships']:
        if col in objects.columns:
            objects[col] = pd.to_numeric(objects[col], errors='coerce')

# OFFICES
filepath = os.path.join(config.RAW_DIR, "offices.csv")
offices = load_csv_safe(filepath)
if offices is not None:
    offices = offices.drop(columns=['id', 'zip_code', 'created_at', 'updated_at'], errors='ignore')
    offices = offices.drop_duplicates()
    offices['state_code'] = offices['state_code'].astype('category')

# PEOPLE
filepath = os.path.join(config.RAW_DIR, "people.csv")
people = load_csv_safe(filepath)
if people is not None:
    people = people.drop(columns=['id'], errors='ignore')
    people = people.drop_duplicates()

# RELATIONSHIPS
filepath = os.path.join(config.RAW_DIR, "relationships.csv")
relationships = load_csv_safe(filepath)
if relationships is not None:
    relationships = relationships.drop(columns=['id', 'sequence', 'created_at', 'updated_at'], errors='ignore')
    relationships = relationships.drop_duplicates()
    relationships['is_past'] = relationships['is_past'].astype('category')

print("\n All datasets loaded successfully")

# =============================================================================
# SUBSETTING MAIN ENTITIES
# =============================================================================

print("SUBSETTING MAIN ENTITIES")

# STARTUPS (Companies)
STARTUPS = objects[
    (objects['entity_type'] == 'Company') & 
    (objects['status'] != '') & 
    (~objects['country_code'].isin(['CSS', 'FST']))
].copy()

STARTUPS = STARTUPS.drop(columns=['entity_id'], errors='ignore')
STARTUPS = STARTUPS.drop_duplicates()
print(f" STARTUPS: {len(STARTUPS):,} companies")

# FINANCIAL_ORG
FINANCIAL_ORG = objects[objects['entity_type'] == 'FinancialOrg'].copy()
FINANCIAL_ORG = FINANCIAL_ORG.drop(columns=[
    'closed_at', 'entity_id', 'parent_id', 'category_code', 'status',
    'funding_rounds', 'funding_total_usd', 'first_funding_at', 'last_funding_at',
    'milestones', 'last_milestone_at', 'first_milestone_at'
], errors='ignore')
print(f" FINANCIAL_ORG: {len(FINANCIAL_ORG):,} organizations")

# PRODUCT
Product = objects[objects['entity_type'] == 'Product'].copy()
Product = Product.drop(columns=[
    'region', 'entity_type', 'investment_rounds', 'invested_companies',
    'funding_rounds', 'first_funding_at', 'last_funding_at',
    'funding_total_usd', 'relationships'
], errors='ignore')
print(f" Product: {len(Product):,} products")

# PERSON
Person = objects[objects['entity_type'] == 'Person'].copy()
print(f" Person: {len(Person):,} people")

# Date range
if 'founded_at' in STARTUPS.columns:
    min_date = STARTUPS['founded_at'].min()
    max_date = STARTUPS['founded_at'].max()
    print(f"\nDataset timespan: {min_date.strftime('%Y-%m-%d')} to {max_date.strftime('%Y-%m-%d')}")

print("\n All entities created successfully")

In [ ]:
# =============================================================================
# USA MARKET ANALYSIS - STARTUP DEEP DIVE
# =============================================================================

print("USA STARTUP MARKET ANALYSIS")

# Filter USA startups
usa_startups = STARTUPS[STARTUPS['country_code'] == 'USA'].copy()

# OVERVIEW
print(" OVERVIEW")
print(f"Total USA startups: {len(usa_startups):,}")
print(f"Percentage of dataset: {100 * len(usa_startups) / len(STARTUPS):.2f}%")
print(f"Date range: {usa_startups['founded_at'].min().strftime('%Y')} - {usa_startups['founded_at'].max().strftime('%Y')}")

# STATUS DISTRIBUTION
print("\n STATUS DISTRIBUTION")

status_usa = usa_startups['status'].value_counts()
status_usa_pct = usa_startups['status'].value_counts(normalize=True) * 100

for status in status_usa.index:
    count = status_usa[status]
    pct = status_usa_pct[status]
    bar = "█" * int(pct / 2)  # Visual bar
    print(f"{status:12s} {count:>7,} ({pct:>5.1f}%) {bar}")

# SUCCESS METRICS
print("\n SUCCESS METRICS")

success_rate = 100 * (status_usa.get('ipo', 0) + status_usa.get('acquired', 0)) / len(usa_startups)
failure_rate = 100 * status_usa.get('closed', 0) / len(usa_startups)
ipo_rate = 100 * status_usa.get('ipo', 0) / len(usa_startups)
acquisition_rate = 100 * status_usa.get('acquired', 0) / len(usa_startups)

print(f"Success rate (IPO + M&A):  {success_rate:>6.2f}%")
print(f"  ↳ IPO rate:              {ipo_rate:>6.2f}%")
print(f"  ↳ Acquisition rate:      {acquisition_rate:>6.2f}%")
print(f"Failure rate (Closed):     {failure_rate:>6.2f}%")
print(f"Still operating:           {100 * status_usa.get('operating', 0) / len(usa_startups):>6.2f}%")

# TOP SECTORS IN USA
print("\n TOP 10 SECTORS IN USA")

usa_sectors = usa_startups['category_code'].value_counts().head(10)
usa_sectors_pct = (usa_sectors / len(usa_startups)) * 100

for i, (sector, count) in enumerate(usa_sectors.items(), 1):
    pct = usa_sectors_pct[sector]
    print(f"{i:2d}. {sector:25s} {count:>6,} ({pct:>5.1f}%)")

# TOP STATES
if 'state_code' in usa_startups.columns:
    print("\n  TOP 10 STATES")

    usa_states = usa_startups['state_code'].value_counts().head(10)
    usa_states_pct = (usa_states / len(usa_startups)) * 100
    
    for i, (state, count) in enumerate(usa_states.items(), 1):
        pct = usa_states_pct[state]
        print(f"{i:2d}. {state:5s} {count:>6,} ({pct:>5.1f}%)")

# FUNDING ANALYSIS
print("\n FUNDING STATISTICS (USA)")

usa_funded = usa_startups[usa_startups['funding_total_usd'] > 0]

if len(usa_funded) > 0:
    print(f"Startups with funding:     {len(usa_funded):>7,} ({100*len(usa_funded)/len(usa_startups):>5.1f}%)")
    print(f"Mean funding:              ${usa_funded['funding_total_usd'].mean()/1e6:>7.1f}M")
    print(f"Median funding:            ${usa_funded['funding_total_usd'].median()/1e6:>7.1f}M")
    print(f"Total capital raised:      ${usa_funded['funding_total_usd'].sum()/1e9:>7.1f}B")
    print(f"Max funding (single co.):  ${usa_funded['funding_total_usd'].max()/1e9:>7.2f}B")

# FUNDING ROUNDS
print("\n FUNDING ROUNDS (USA)")

rounds_stats = usa_startups['funding_rounds'].describe()
print(f"Mean rounds:               {rounds_stats['mean']:>7.2f}")
print(f"Median rounds:             {rounds_stats['50%']:>7.0f}")
print(f"Max rounds:                {rounds_stats['max']:>7.0f}")

# Companies with 5+ rounds
high_rounds = (usa_startups['funding_rounds'] >= 5).sum()
print(f"Companies with 5+ rounds:  {high_rounds:>7,} ({100*high_rounds/len(usa_startups):>5.1f}%)")

# TEMPORAL ANALYSIS
print("\n FOUNDATION TIMELINE")

usa_startups['founded_year'] = usa_startups['founded_at'].dt.year
yearly = usa_startups['founded_year'].value_counts().sort_index()

# Show recent years
recent_years = yearly.tail(10)
print("Recent years (last 10):")
for year, count in recent_years.items():
    print(f"  {year}: {count:>5,} startups")

# SECTOR SUCCESS RATES
print("\n TOP 10 SECTORS BY IPO RATE (USA)")

usa_sector_success = usa_startups.groupby('category_code').agg({
    'id': 'count',
    'status': lambda x: (x == 'ipo').sum()
}).rename(columns={'id': 'total', 'status': 'ipos'})

usa_sector_success = usa_sector_success[usa_sector_success['total'] >= 50]  # Min 50 companies
usa_sector_success['ipo_rate'] = 100 * usa_sector_success['ipos'] / usa_sector_success['total']
usa_sector_success = usa_sector_success.sort_values('ipo_rate', ascending=False).head(10)

for sector, row in usa_sector_success.iterrows():
    print(f"{sector:25s} {row['ipo_rate']:>5.1f}% ({int(row['ipos']):>3}/{int(row['total']):>4})")

# VISUALIZATION

print("GENERATING VISUALIZATIONS")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Status distribution
status_usa.plot(kind='bar', ax=axes[0,0], color='steelblue', alpha=0.8)
axes[0,0].set_title('USA Startups - Status Distribution', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('Status')
axes[0,0].set_ylabel('Count')
axes[0,0].grid(axis='y', alpha=0.3)
axes[0,0].set_xticklabels(axes[0,0].get_xticklabels(), rotation=0)

# 2. Top 15 sectors
top_15_sectors = usa_startups['category_code'].value_counts().head(15).sort_values()
top_15_sectors.plot(kind='barh', ax=axes[0,1], color='coral')
axes[0,1].set_title('Top 15 Sectors in USA', fontsize=14, fontweight='bold')
axes[0,1].set_xlabel('Number of Startups')
axes[0,1].grid(axis='x', alpha=0.3)

# 3. Funding distribution (log scale)
usa_funded_log = np.log10(usa_funded['funding_total_usd'] + 1)
axes[1,0].hist(usa_funded_log, bins=50, color='green', alpha=0.7, edgecolor='white')
axes[1,0].set_title('Funding Distribution (Log Scale)', fontsize=14, fontweight='bold')
axes[1,0].set_xlabel('Log10(Funding USD)')
axes[1,0].set_ylabel('Frequency')
axes[1,0].axvline(usa_funded_log.median(), color='red', linestyle='--', 
                  label=f'Median: ${10**usa_funded_log.median()/1e6:.1f}M')
axes[1,0].legend()
axes[1,0].grid(axis='y', alpha=0.3)

# 4. Foundation timeline
yearly_plot = usa_startups['founded_year'].value_counts().sort_index()
yearly_plot.plot(kind='line', ax=axes[1,1], color='purple', linewidth=2)
axes[1,1].set_title('USA Startups Founded per Year', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('Year')
axes[1,1].set_ylabel('Number of Startups Founded')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(" Analysis complete")

### 2.5 Dataset Exploration: Jay Ritter IPO

Let's analyze the Jay Ritter database to understand:
1. Data structure and quality
2. Temporal distribution of IPOs
3. Proportion of VC-backed IPOs
4. Useful variables for matching with Crunchbase

In [ ]:
# =============================================================================
# 2.5 ESPLORAZIONE JAY RITTER IPO
# =============================================================================

if ritter is not None:

    print("JAY RITTER IPO DATABASE - STRUCTURE")


    print(f"\n Shape: {ritter.shape}")
    print(f"\n Colonne: {ritter.columns.tolist()}")

    # Info
    ritter.info()

    # Prime righe
    print(ritter.head().to_string())

# %%
# Statistiche descrittive Ritter
if ritter is not None:

    print("JAY RITTER - DESCRIPTIVE STATISTICS")

    # Pulizia colonne
    # 'offer date' è in formato YYYYMMDD
    ritter["ipo_date"] = pd.to_datetime(
        ritter["offer date"].astype(str), format="%Y%m%d", errors="coerce"
    )
    ritter["ipo_year"] = ritter["ipo_date"].dt.year

    # 'Founding' ha -99 per missing
    ritter["founding_year_clean"] = ritter["Founding"].replace(-99, np.nan)
    ritter.loc[ritter["founding_year_clean"] < 1800, "founding_year_clean"] = np.nan

    # Calcola time-to-IPO
    ritter["time_to_ipo"] = ritter["ipo_year"] - ritter["founding_year_clean"]

    print("\n Distribution VC Status:")
    print(ritter["VC"].value_counts())
    print("\n   Legend: 0=Non-VC, 1=VC-backed, 2=Growth Capital")

    # Visualizzazioni
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    # IPO per anno
    ax1 = axes[0, 0]
    ritter["ipo_year"].value_counts().sort_index().plot(ax=ax1, color="steelblue")
    ax1.set_title("IPO per Anno (Jay Ritter)", fontsize=12)
    ax1.set_xlabel("Year")
    ax1.set_ylabel("Number IPO")

    # IPO per decade
    ax2 = axes[0, 1]
    ritter["decade"] = (ritter["ipo_year"] // 10) * 10
    ritter["decade"].value_counts().sort_index().plot(kind="bar", ax=ax2, color="coral")
    ax2.set_title("IPO per Decade", fontsize=12)
    ax2.set_xlabel("Decade")
    ax2.set_ylabel("Number IPO")
    ax2.set_xticklabels(
        [f"{int(d)}s" for d in sorted(ritter["decade"].dropna().unique())], rotation=45
    )

    # Time to IPO distribution
    ax3 = axes[1, 0]
    valid_time = ritter["time_to_ipo"][
        (ritter["time_to_ipo"] > 0) & (ritter["time_to_ipo"] < 100)
    ]
    valid_time.hist(bins=50, ax=ax3, color="green", alpha=0.7, edgecolor="white")
    ax3.axvline(
        valid_time.median(),
        color="red",
        linestyle="--",
        label=f"Median: {valid_time.median():.1f} anni",
    )
    ax3.set_title("Distribution Time-to-IPO (Ritter)", fontsize=12)
    ax3.set_xlabel("Years from founding to IPO")
    ax3.set_ylabel("Frequency")
    ax3.legend()

    # VC vs Non-VC time to IPO
    ax4 = axes[1, 1]
    vc_backed = ritter[ritter["VC"].isin([1, 2])]["time_to_ipo"]
    non_vc = ritter[ritter["VC"] == 0]["time_to_ipo"]

    ax4.boxplot([vc_backed.dropna(), non_vc.dropna()], labels=["VC-Backed", "Non-VC"])
    ax4.set_title("Time-to-IPO: VC vs Non-VC", fontsize=12)
    ax4.set_ylabel("Years")

    plt.tight_layout()
    plt.show()

    # Statistiche
    print(f"\n Time-to-IPO Statistics:")
    print(f"   Overall median: {valid_time.median():.1f} anni")
    print(
        f"   VC-backed median: {vc_backed[(vc_backed > 0) & (vc_backed < 100)].median():.1f} anni"
    )
    print(
        f"   Non-VC median: {non_vc[(non_vc > 0) & (non_vc < 100)].median():.1f} anni"
    )

In [ ]:
# =============================================================================
# 2.7 MATCHING CRUNCHBASE ↔ JAY RITTER
# =============================================================================

def normalize_company_name(name):
    """Normalizes a company name for matching.

    Conservative strategy:
    - Lowercase
    - Remove legal suffixes
    - Remove punctuation
    - Strip spaces

    Parameters
    ----------
    name : str
        Original name

    Returns
    -------
    str or None
        Normalized name

    """
    if pd.isna(name):
        return None

    name = str(name).lower().strip()

    # Rimuovi suffissi legali (ordine importante: più lunghi prima)
    suffixes = [
        " incorporated",
        " corporation",
        " limited",
        " company",
        " inc.",
        " inc",
        " corp.",
        " corp",
        " llc",
        " ltd.",
        " ltd",
        " co.",
        " co",
        " plc",
        " sa",
        " ag",
        " gmbh",
        " nv",
        " bv",
        ", inc.",
        ", inc",
        ", corp.",
        ", corp",
        ", llc",
        ", ltd",
    ]

    for suffix in suffixes:
        if name.endswith(suffix):
            name = name[: -len(suffix)]

    # Rimuovi punteggiatura
    name = re.sub(r"[^\w\s]", "", name)

    # Rimuovi spazi multipli e strip
    name = re.sub(r"\s+", " ", name).strip()

    return name if name else None


# Test normalizzazione
test_names = [
    "Apple Inc.",
    "Microsoft Corporation",
    "Google, Inc.",
    "FACEBOOK INC",
    "Tesla Motors, Inc.",
]

print("NAMES STANDARDIZATION TEST")

for name in test_names:
    print(f"   '{name}' → '{normalize_company_name(name)}'")

# =============================================================================
# PREPARAZIONE DATI PER MATCHING
# =============================================================================

print("PREPARATION MATCHING CRUNCHBASE ↔ RITTER")


# Prepara Crunchbase IPOs
if crunchbase.get("ipos") is not None and crunchbase.get("objects") is not None:
    # Unisci IPO con objects per avere i nomi
    ipos_with_names = crunchbase["ipos"].merge(
        crunchbase["objects"][["id", "name", "country_code"]],
        left_on="object_id",
        right_on="id",
        how="left",
    )

    ipos_with_names["name_normalized"] = ipos_with_names["name"].apply(
        normalize_company_name
    )
    ipos_with_names["ipo_year"] = pd.to_datetime(
        ipos_with_names["public_at"], errors="coerce"
    ).dt.year

    # Filtra solo USA per matching con Ritter (che è solo USA)
    ipos_usa = ipos_with_names[ipos_with_names["country_code"] == "USA"].copy()

    print(
        f" Crunchbase IPO prepared: {len(ipos_with_names):,} totals, {len(ipos_usa):,} USA"
    )

# Prepara Ritter
if ritter is not None:
    ritter["name_normalized"] = ritter["IPO name"].apply(normalize_company_name)
    ritter["ipo_year"] = pd.to_datetime(
        ritter["offer date"].astype(str), format="%Y%m%d", errors="coerce"
    ).dt.year

    print(f" Ritter IPO preparate: {len(ritter):,}")

# =============================================================================
# ESECUZIONE MATCHING CONSERVATIVO
# =============================================================================

print("MATCHING CONSERVATIVE: EXACT NAME + YEAR (±1)")

if "ipos_usa" in dir() and ritter is not None:
    # Strategia 1: Exact match su nome normalizzato + stesso anno
    matched_exact = pd.merge(
        ipos_usa[["object_id", "name", "name_normalized", "ipo_year", "stock_symbol"]],
        ritter[["IPO name", "name_normalized", "ipo_year", "Ticker", "VC", "Founding"]],
        on=["name_normalized", "ipo_year"],
        how="inner",
        suffixes=("_cb", "_ritter"),
    )

    print(f"\n Match esatti (nome + anno): {len(matched_exact):,}")

    # Strategia 2: Exact match su nome + anno ±1
    matches_year_flex = []

    for _, row_cb in ipos_usa.iterrows():
        if pd.isna(row_cb["name_normalized"]) or pd.isna(row_cb["ipo_year"]):
            continue

        # Cerca in Ritter con tolleranza ±1 anno
        mask = (ritter["name_normalized"] == row_cb["name_normalized"]) & (
            abs(ritter["ipo_year"] - row_cb["ipo_year"]) <= 1
        )

        ritter_matches = ritter[mask]

        if len(ritter_matches) == 1:  # Solo match univoci
            matches_year_flex.append(
                {
                    "object_id": row_cb["object_id"],
                    "name_cb": row_cb["name"],
                    "name_ritter": ritter_matches.iloc[0]["IPO name"],
                    "year_cb": row_cb["ipo_year"],
                    "year_ritter": ritter_matches.iloc[0]["ipo_year"],
                    "vc_ritter": ritter_matches.iloc[0]["VC"],
                    "founding_ritter": ritter_matches.iloc[0]["Founding"],
                    "match_type": "exact_name_year_flex",
                }
            )

    matched_flex = pd.DataFrame(matches_year_flex)

    print(f" Match with year tolerance ±1: {len(matched_flex):,}")

    # Statistiche matching
    total_cb_usa = len(ipos_usa)
    total_ritter = len(ritter)

    print(f"\n STATISTICS MATCHING:")
    print(f"Crunchbase IPO USA: {total_cb_usa:,}")
    print(f"Jay Ritter IPO: {total_ritter:,}")
    print(f"Match found: {len(matched_flex):,}")
    print(f"Match rate (su CB): {100 * len(matched_flex) / total_cb_usa:.1f}%")
    print(f"Match rate (su Ritter): {100 * len(matched_flex) / total_ritter:.1f}%")

    # Sample dei match
    if len(matched_flex) > 0:
        print(f"\n Sample Matches:")
        print(matched_flex.head().to_string())

### 2.2 Build Final Integrated Dataset

In [ ]:
# =============================================================================
# BUILDING FINAL DATASET
# =============================================================================

print("BUILDING FINAL DATASET")

# Start with STARTUPS
finale = STARTUPS.copy()

# Merge with IPOs
if ipos is not None:
    finale = finale.merge(
        ipos[['object_id', 'valuation_amount']],
        left_on='id', right_on='object_id', how='left'
    ).drop(columns=['object_id'], errors='ignore')

# Drop redundant columns
finale = finale.drop(columns=[
    'homepage_url', 'parent_id', 'entity_type', 'short_description',
    'description', 'tag_list'
], errors='ignore')

print(f"{finale.shape}")

In [ ]:
# Aggregate funding by round type
if funding_rounds is not None:
    rounds_data = finale[['id']].merge(
        funding_rounds[['object_id', 'funding_round_type', 'raised_amount_usd']],
        left_on='id', right_on='object_id', how='left'
    ).drop(columns=['object_id'])
    
    # Pivot to get funding per round type
    rounds_pivot = rounds_data.pivot_table(
        index='id',
        columns='funding_round_type',
        values='raised_amount_usd',
        aggfunc='sum',
        fill_value=0
    )
    
    # Ensure all round types exist
    round_types = ['angel', 'crowdfunding', 'other', 'post-ipo', 
                   'private-equity', 'series-a', 'series-b', 'series-c+', 'venture']
    for rt in round_types:
        if rt not in rounds_pivot.columns:
            rounds_pivot[rt] = 0
    
    # Rename columns
    rounds_pivot = rounds_pivot.rename(columns={
        'post-ipo': 'post_ipo',
        'private-equity': 'private_equity',
        'series-a': 'series_a',
        'series-b': 'series_b',
        'series-c+': 'series_c'
    })
    
    # Merge with finale
    finale = finale.merge(rounds_pivot, left_on='id', right_index=True, how='left')
    for col in rounds_pivot.columns:
        finale[col] = finale[col].fillna(0)

print(f"{finale.shape}")

In [ ]:
# Number of acquisitions made
if acquisitions is not None:
    num_acq = acquisitions.groupby('acquiring_object_id').size().reset_index(name='num_acquisizioni_effettuate')
    finale = finale.merge(num_acq, left_on='id', right_on='acquiring_object_id', how='left')
    finale['num_acquisizioni_effettuate'] = finale['num_acquisizioni_effettuate'].fillna(0).astype(int)
    finale = finale.drop(columns=['acquiring_object_id'], errors='ignore')

print(f"{finale.shape}")

In [ ]:
# Funding sources
if investments is not None:
    # Financial organizations
    fin_org_inv = investments[
        investments['investor_object_id'].isin(FINANCIAL_ORG['id'])
    ].groupby('funded_object_id')['investor_object_id'].nunique().reset_index(name='fin_org_financed')
    
    finale = finale.merge(fin_org_inv, left_on='id', right_on='funded_object_id', how='left')
    finale['fin_org_financed'] = (finale['fin_org_financed'] > 0).astype(int)
    finale = finale.drop(columns=['funded_object_id'], errors='ignore')
    
    # People
    person_inv = investments[
        investments['investor_object_id'].isin(Person['id'])
    ].groupby('funded_object_id').size().reset_index(name='person_financed')
    
    finale = finale.merge(person_inv, left_on='id', right_on='funded_object_id', how='left')
    finale['person_financed'] = (finale['person_financed'] > 0).astype(int)
    finale = finale.drop(columns=['funded_object_id'], errors='ignore')
    
    # Startups
    startup_inv = investments[
        investments['investor_object_id'].isin(STARTUPS['id'])
    ].groupby('funded_object_id').size().reset_index(name='startup_financed')
    
    finale = finale.merge(startup_inv, left_on='id', right_on='funded_object_id', how='left')
    finale['startup_financed'] = (finale['startup_financed'] > 0).astype(int)
    finale = finale.drop(columns=['funded_object_id'], errors='ignore')

print(f"{finale.shape}")

In [ ]:
# Number of products
num_prod = Product[
    Product['parent_id'].notna() & (Product['parent_id'] != '')
].groupby('parent_id').size().reset_index(name='num_prodotti')

finale = finale.merge(num_prod, left_on='id', right_on='parent_id', how='left')
finale['num_prodotti'] = finale['num_prodotti'].fillna(0).astype(int)
finale = finale.drop(columns=['parent_id'], errors='ignore')

print(f"{finale.shape}")

In [ ]:
# Load Jay Ritter IPO dataset
ritter_path = os.path.join(config.RAW_DIR, "IPO-age.xlsx")

if os.path.exists(ritter_path):
    print(f"Found Ritter file: {ritter_path}")
    
    # Load Excel file
    ritter = pd.read_excel(ritter_path, sheet_name=0)
    print(f"Ritter loaded: {ritter.shape}")
    
    # --- DATA CLEANING ---
    
    # 1. Parse offer date (format: YYYYMMDD → extract year)
    if 'offer date' in ritter.columns:
        ritter['offer_date_str'] = ritter['offer date'].astype(str)
        ritter['ipo_year'] = ritter['offer_date_str'].str[:4].astype(float)
    elif 'Offer Date' in ritter.columns:
        ritter['Offer Date'] = pd.to_datetime(ritter['Offer Date'], errors='coerce')
        ritter['ipo_year'] = ritter['Offer Date'].dt.year
    else:
        print("⚠ Warning: Date column not found, trying to infer...")
        date_col = [c for c in ritter.columns if 'date' in c.lower()][0]
        ritter['ipo_year'] = pd.to_datetime(ritter[date_col], errors='coerce').dt.year
    
    # 2. Convert VC/Internet to binary
    if 'VC' in ritter.columns:
        ritter['is_vc_backed'] = (ritter['VC'] == 'Y').astype(int)
    
    if 'Internet' in ritter.columns:
        ritter['is_tech'] = (ritter['Internet'] == 'Y').astype(int)
    
    # 3. Calculate time-to-IPO
    if 'Founding' in ritter.columns:
        ritter['time_to_ipo'] = ritter['ipo_year'] - ritter['Founding']
    
    # 4. Filter outliers
    ritter_clean = ritter[
        (ritter['ipo_year'] >= 1975) & 
        (ritter['ipo_year'] <= 2024)
    ].copy()
    
    if 'time_to_ipo' in ritter_clean.columns:
        ritter_clean = ritter_clean[
            (ritter_clean['time_to_ipo'] >= 0) & 
            (ritter_clean['time_to_ipo'] <= 100)
        ]
    
    print(f"Ritter cleaned: {ritter_clean.shape}")
    print(f"Date range: {ritter_clean['ipo_year'].min():.0f} - {ritter_clean['ipo_year'].max():.0f}")
    
    # --- AGGREGATE BY YEAR ---
    
    print("AGGREGATING MARKET CONDITIONS BY YEAR")
    
    market_conditions = ritter_clean.groupby('ipo_year').agg({
        'IPO name': 'count',  # Total IPOs
    }).rename(columns={'IPO name': 'ipo_count_total'})
    
    # Add VC and tech counts
    if 'is_vc_backed' in ritter_clean.columns:
        market_conditions['ipo_count_vc'] = ritter_clean.groupby('ipo_year')['is_vc_backed'].sum()
        market_conditions['vc_share_pct'] = (
            market_conditions['ipo_count_vc'] / market_conditions['ipo_count_total']
        ) * 100
    
    if 'is_tech' in ritter_clean.columns:
        market_conditions['ipo_count_tech'] = ritter_clean.groupby('ipo_year')['is_tech'].sum()
        market_conditions['tech_share_pct'] = (
            market_conditions['ipo_count_tech'] / market_conditions['ipo_count_total']
        ) * 100
    
    # Add time-to-IPO statistics
    if 'time_to_ipo' in ritter_clean.columns:
        market_conditions['time_to_ipo_mean'] = ritter_clean.groupby('ipo_year')['time_to_ipo'].mean()
        market_conditions['time_to_ipo_median'] = ritter_clean.groupby('ipo_year')['time_to_ipo'].median()
    
    # Market heat index (normalized by historical average)
    historical_avg = market_conditions['ipo_count_total'].mean()
    market_conditions['market_heat'] = market_conditions['ipo_count_total'] / historical_avg
    
    # Moving average (3-year smoothing)
    market_conditions['ipo_count_ma3'] = market_conditions['ipo_count_total'].rolling(
        window=3, center=True, min_periods=1
    ).mean()
    
    # Market cycle labels (manual categorization)
    def assign_market_cycle(year):
        if year in range(1999, 2001):
            return 'boom'
        elif year in range(2001, 2003):
            return 'bust'
        elif year in range(2008, 2010):
            return 'bust'
        elif year in range(2020, 2022):
            return 'boom'
        elif year in range(2022, 2025):
            return 'contraction'
        else:
            return 'normal'
    
    market_conditions['market_cycle'] = market_conditions.index.map(assign_market_cycle)
    
    market_conditions = market_conditions.reset_index()
    
    print(f"Market conditions aggregated: {market_conditions.shape}")
    print(f"Years covered: {market_conditions['ipo_year'].min():.0f} - {market_conditions['ipo_year'].max():.0f}")
    
    # Show sample
    print("\nSample (recent years):")
    print(market_conditions.tail(10)[['ipo_year', 'ipo_count_total', 'market_heat', 'market_cycle']])
    
    # --- MERGE WITH FINALE ---
    
    print("MERGING MARKET CONDITIONS WITH STARTUPS")
    
    # Extract first funding year from finale
    if 'first_funding_at' in finale.columns:
        # Use already existing first_funding_at if available
        finale['first_funding_year'] = pd.to_datetime(finale['first_funding_at']).dt.year
    else:
        # Fallback: use founded_at
        finale['first_funding_year'] = finale['founded_at'].dt.year
    
    # Merge
    finale = finale.merge(
        market_conditions,
        left_on='first_funding_year',
        right_on='ipo_year',
        how='left'
    )
    
    # Drop duplicate year column
    finale = finale.drop(columns=['ipo_year'], errors='ignore')
    
    print(f"Merge complete: {finale.shape}")
    
    # Check merge quality
    merged_pct = 100 * finale['ipo_count_total'].notna().sum() / len(finale)
    print(f"Startups with market data: {merged_pct:.1f}%")
    
    # Show new columns
    new_cols = [c for c in market_conditions.columns if c != 'ipo_year']
    print(f"\nNew columns added ({len(new_cols)}):")
    for col in new_cols[:5]:  # Show first 5
        print(f"• {col}")
    if len(new_cols) > 5:
        print(f"and {len(new_cols)-5} more")
    
    print(f"\n{finale.shape}")
    
else:
    print(f"Ritter file not found: {ritter_path}")
    print("Download from: https://site.warrington.ufl.edu/ritter/files/IPO-age.xlsx")

In [ ]:
has_market_data = finale['ipo_count_total'].notna()
no_market_data = ~has_market_data

print(f"Startups WITH market data:    {has_market_data.sum():>8,} ({100*has_market_data.mean():>5.1f}%)")
print(f"Startups WITHOUT market data:  {no_market_data.sum():>8,} ({100*no_market_data.mean():>5.1f}%)")

# 2. Why missing? Analyze first_funding_year distribution
print("\nFIRST FUNDING YEAR DISTRIBUTION")

# Overall distribution
funding_year_stats = finale['first_funding_year'].describe()
print(f"Min year:    {funding_year_stats['min']:.0f}")
print(f"25% year:    {funding_year_stats['25%']:.0f}")
print(f"Median year: {funding_year_stats['50%']:.0f}")
print(f"75% year:    {funding_year_stats['75%']:.0f}")
print(f"Max year:    {funding_year_stats['max']:.0f}")

# Ritter coverage
ritter_min_year = 1975
ritter_max_year = 2024

print(f"\nRitter coverage: {ritter_min_year} - {ritter_max_year}")

# Count startups by year range
before_ritter = (finale['first_funding_year'] < ritter_min_year).sum()
in_ritter_range = (
    (finale['first_funding_year'] >= ritter_min_year) & 
    (finale['first_funding_year'] <= ritter_max_year)
).sum()
after_ritter = (finale['first_funding_year'] > ritter_max_year).sum()
missing_year = finale['first_funding_year'].isna().sum()

print(f"\nStartups by year range:")
print(f"Before 1975:{before_ritter:>8,} ({100*before_ritter/len(finale):>5.1f}%)")
print(f"1975-2024 (Ritter):{in_ritter_range:>8,} ({100*in_ritter_range/len(finale):>5.1f}%)")
print(f"After 2024:{after_ritter:>8,} ({100*after_ritter/len(finale):>5.1f}%)")
print(f"Missing year:{missing_year:>8,} ({100*missing_year/len(finale):>5.1f}%)")

# 3. Match quality for in-range startups
in_range = finale[
    (finale['first_funding_year'] >= ritter_min_year) & 
    (finale['first_funding_year'] <= ritter_max_year)
].copy()

if len(in_range) > 0:
    matched = in_range['ipo_count_total'].notna().sum()
    match_rate = 100 * matched / len(in_range)
    
    print(f"Startups in range (1975-2024):{len(in_range):,}")
    print(f"Successfully matched:{matched:,} ({match_rate:.1f}%)")
    
    if match_rate < 100:
        print(f"\n{100-match_rate:.1f}% in-range startups have no match!")
        print("Likely cause: first_funding_year has NaN values")

# 4. Sample comparison: WITH vs WITHOUT market data
with_data = finale[has_market_data].copy()
without_data = finale[no_market_data].copy()

if len(with_data) > 0 and len(without_data) > 0:
    print(f"\nIPO Rate:")
    print(f"WITH market data:{100 * (with_data['status'] == 'ipo').mean():>5.2f}%")
    print(f"WITHOUT market data:{100 * (without_data['status'] == 'ipo').mean():>5.2f}%")
    
    print(f"\nAcquisition Rate:")
    print(f"WITH market data:{100 * (with_data['status'] == 'acquired').mean():>5.2f}%")
    print(f"WITHOUT market data:{100 * (without_data['status'] == 'acquired').mean():>5.2f}%")
    
    print(f"\nMean funding (for funded companies):")
    with_funding = with_data[with_data['funding_total_usd'] > 0]['funding_total_usd'].mean()
    without_funding = without_data[without_data['funding_total_usd'] > 0]['funding_total_usd'].mean()
    print(f"WITH market data:${with_funding/1e6:>7.1f}M")
    print(f"WITHOUT market data:${without_funding/1e6:>7.1f}M")

# 5. Show examples with market data
sample_with = with_data[with_data['ipo_count_total'].notna()].head(3)

for idx, row in sample_with.iterrows():
    print(f"\nStartup ID: {row.get('id', 'N/A')}")
    print(f"First funding year:{row['first_funding_year']:.0f}")
    print(f"IPOs that year:{row['ipo_count_total']:.0f}")
    print(f"Market heat:{row['market_heat']:.2f}")
    print(f"Market cycle:{row['market_cycle']}")
    print(f"Status:{row['status']}")

# 6. Recommendation
coverage_pct = 100 * has_market_data.mean()

if coverage_pct < 30:
    print("\nLOW COVERAGE (<30%)")
    print("Consider using founded_year instead of first_funding_year")
    print("Or extrapolate market conditions for years outside Ritter range")
elif coverage_pct < 70:
    print("\nMEDIUM COVERAGE (30-70%)")
    print("Current approach OK for analysis")
    print("Consider imputation for missing values in modeling")
else:
    print("\nGOOD COVERAGE (>70%)")
    print("Market conditions successfully merged!")

print(f"\nCurrent coverage: {coverage_pct:.1f}%")

In [ ]:
# Cleaning
finale = finale.drop(columns=[
    'first_milestone_at', 'last_milestone_at', 'last_funding_at',
    'first_funding_at', 'name', 'city', 'region', 'closed_at'
], errors='ignore')

finale = finale.drop_duplicates()

# Type conversions
for col in ['category_code', 'status', 'country_code']:
    if col in finale.columns:
        finale[col] = finale[col].astype('category')

for col in ['fin_org_financed', 'person_financed', 'startup_financed']:
    if col in finale.columns:
        finale[col] = finale[col].astype(int)

# Log transformation for funding
if 'funding_total_usd' in finale.columns:
    finale['log_fund_tot'] = finale['funding_total_usd'].apply(
        lambda x: 0 if pd.isna(x) or x == 0 else np.log(x)
    )

print(f"\nFinal dataset: {finale.shape}")
print(f"\nColumns: {list(finale.columns[:15])}...")

In [ ]:
# ---------------------------------------------------------------------------
# 1. Merge IPO dates
# ---------------------------------------------------------------------------

if 'ipos' in locals() or 'ipos' in globals():
    # Prepare IPO dates
    ipos_clean = ipos[['object_id', 'public_at']].copy()
    ipos_clean = ipos_clean.drop_duplicates(subset=['object_id'])
    
    print(f"  IPO records available: {len(ipos_clean):,}")
    
    # Count before merge
    n_ipo_before = (finale['status'] == 'ipo').sum()
    
    # Merge
    finale = finale.merge(
        ipos_clean,
        left_on='id',
        right_on='object_id',
        how='left'
    )
    
    # Drop duplicate column
    finale = finale.drop(columns=['object_id'], errors='ignore')
    
    # Check merge quality
    n_matched = finale[finale['status'] == 'ipo']['public_at'].notna().sum()
    
    print(f"IPO companies in dataset: {n_ipo_before:,}")
    print(f"Dates matched: {n_matched:,} ({100*n_matched/n_ipo_before:.1f}%)")
    
    if n_matched < n_ipo_before:
        print(f"Missing dates: {n_ipo_before - n_matched:,} (will be handled in Notebook 2)")
else:
    print(f"'ipos' not found - creating empty column")
    finale['public_at'] = pd.NaT

# ---------------------------------------------------------------------------
# 2. Merge M&A dates
# ---------------------------------------------------------------------------

if 'acquisitions' in locals() or 'acquisitions' in globals():
    # Prepare M&A dates
    acq_clean = acquisitions[['acquired_object_id', 'acquired_at']].copy()
    acq_clean = acq_clean.drop_duplicates(subset=['acquired_object_id'])
    
    print(f"M&A records available: {len(acq_clean):,}")
    
    # Count before merge
    n_ma_before = (finale['status'] == 'acquired').sum()
    
    # Merge
    finale = finale.merge(
        acq_clean,
        left_on='id',
        right_on='acquired_object_id',
        how='left'
    )
    
    # Drop duplicate column
    finale = finale.drop(columns=['acquired_object_id'], errors='ignore')
    
    # Check merge quality
    n_matched = finale[finale['status'] == 'acquired']['acquired_at'].notna().sum()
    
    print(f"M&A companies in dataset: {n_ma_before:,}")
    print(f"Dates matched: {n_matched:,} ({100*n_matched/n_ma_before:.1f}%)")
    
    if n_matched < n_ma_before:
        print(f"Missing dates: {n_ma_before - n_matched:,} (will be handled in Notebook 2)")
else:
    print(f"'acquisitions' not found - creating empty column")
    finale['acquired_at'] = pd.NaT

# ---------------------------------------------------------------------------
# 3. Summary
# ---------------------------------------------------------------------------

print(f"\nDataset shape: {finale.shape}")
print(f"\nNew columns added:")
print(f"  - public_at:   {finale['public_at'].notna().sum():,} non-null ({100*finale['public_at'].notna().mean():.1f}%)")
print(f"  - acquired_at: {finale['acquired_at'].notna().sum():,} non-null ({100*finale['acquired_at'].notna().mean():.1f}%)")

In [ ]:
# -----------------------------------------------------------------------------
# Step 1: Keep only startups with real first_funding_year
# -----------------------------------------------------------------------------

print(f"\nDataset before filtering:")
print(f"Total startups:{len(finale):,}")
print(f"With first_funding_year:{finale['first_funding_year'].notna().sum():,} ({100*finale['first_funding_year'].notna().mean():.1f}%)")

# Filter
finale_real = finale[finale['first_funding_year'].notna()].copy()

print(f"\nFiltered to real funding data only:")
print(f"Kept:{len(finale_real):,} startups")
print(f"Removed:{len(finale) - len(finale_real):,} startups without funding year")

# -----------------------------------------------------------------------------
# Step 2: Keep only years in Ritter range (1975-2024)
# -----------------------------------------------------------------------------

print(f"\nFiltering for Ritter range (1975-2024)...")

before_1975 = (finale_real['first_funding_year'] < 1975).sum()
after_2024 = (finale_real['first_funding_year'] > 2024).sum()

finale_real = finale_real[
    (finale_real['first_funding_year'] >= 1975) &
    (finale_real['first_funding_year'] <= 2013)
].copy()

if before_1975 + after_2024 > 0:
    print(f"Removed{before_1975:,} startups (funding < 1975)")
    print(f"Removed{after_2024:,} startups (funding > 2024)")
print(f"Kept:{len(finale_real):,} startups")

# -----------------------------------------------------------------------------
# Step 3: Verify 100% market data coverage
# -----------------------------------------------------------------------------

print(f"\nVerifying market data coverage...")

coverage = 100 * finale_real['ipo_count_total'].notna().mean()
missing = finale_real['ipo_count_total'].isna().sum()

print(f"Market data coverage:{coverage:.1f}%")

if missing > 0:
    print(f"Removing{missing:,} startups with missing market data...")
    finale_real = finale_real[finale_real['ipo_count_total'].notna()].copy()
    print(f"Final:{len(finale_real):,} startups")
else:
    print(f"Perfect! 100% coverage")

# -----------------------------------------------------------------------------
# Step 4: Filter USA only
# -----------------------------------------------------------------------------

finale_usa = finale_real[finale_real['country_code'] == 'USA'].copy()

print(f"Global (with real funding):{len(finale_real):,}")
print(f"USA only:{len(finale_usa):,} ({100*len(finale_usa)/len(finale_real):.1f}%)")

# -----------------------------------------------------------------------------
# FINAL DATASET SUMMARY
# -----------------------------------------------------------------------------

print(f"\nSample Size:{len(finale_usa):,} startups")

print(f"\nExit Distribution:")
status_counts = finale_usa['status'].value_counts()
for status in ['ipo', 'acquired', 'closed', 'operating']:
    if status in status_counts.index:
        count = status_counts[status]
        pct = 100 * count / len(finale_usa)
        print(f"{status:10s} {count:>6,} ({pct:>5.1f}%)")

print(f"\nFunding Year Range:")
print(f"Min:{finale_usa['first_funding_year'].min():.0f}")
print(f"25%:{finale_usa['first_funding_year'].quantile(0.25):.0f}")
print(f"Median:{finale_usa['first_funding_year'].median():.0f}")
print(f"75%:{finale_usa['first_funding_year'].quantile(0.75):.0f}")
print(f"Max:{finale_usa['first_funding_year'].max():.0f}")

print(f"\nFunding Statistics:")
funded_usa = finale_usa[finale_usa['funding_total_usd'] > 0]
if len(funded_usa) > 0:
    print(f"Startups with funding:{len(funded_usa):,} ({100*len(funded_usa)/len(finale_usa):.1f}%)")
    print(f"Mean:${funded_usa['funding_total_usd'].mean()/1e6:.1f}M")
    print(f"Median:${funded_usa['funding_total_usd'].median()/1e6:.1f}M")

print(f"\nTop 5 Sectors:")
top_sectors = finale_usa['category_code'].value_counts().head(5)
for sector, count in top_sectors.items():
    pct = 100 * count / len(finale_usa)
    print(f"{sector:20s} {count:>6,} ({pct:>5.1f}%)")

print(f"\nMarket Conditions Distribution:")
for cycle in ['boom', 'normal', 'bust', 'contraction']:
    count = (finale_usa['market_cycle'] == cycle).sum()
    if count > 0:
        pct = 100 * count / len(finale_usa)
        print(f"{cycle:15s} {count:>6,} ({pct:>5.1f}%)")

# =============================================================================
# CREATE HOT/COLD MARKET INDICATOR
# =============================================================================

# Calculate threshold (top 25% of IPO activity)
q75 = market_conditions['ipo_count_total'].quantile(0.75)
finale_usa['hot_ipo_market'] = (finale_usa['ipo_count_total'] >= q75).astype(int)

hot_count = (finale_usa['hot_ipo_market'] == 1).sum()
cold_count = (finale_usa['hot_ipo_market'] == 0).sum()

print(f"Hot market threshold: {q75:.0f} IPOs/year")
print(f"\nMarket Hotness Distribution:")
print(f"Hot (top 25%):{hot_count:>6,} ({100*hot_count/len(finale_usa):>5.1f}%)")
print(f"Cold (bottom 75%):{cold_count:>6,} ({100*cold_count/len(finale_usa):>5.1f}%)")

# =============================================================================
# MARKET IMPACT ANALYSIS
# =============================================================================

print("\nEXIT RATES BY MARKET HOTNESS")

for hot_val in [1, 0]:
    label = "Hot market" if hot_val == 1 else "Cold market"
    subset = finale_usa[finale_usa['hot_ipo_market'] == hot_val]
    
    if len(subset) > 0:
        ipo_rate = 100 * (subset['status'] == 'ipo').sum() / len(subset)
        acq_rate = 100 * (subset['status'] == 'acquired').sum() / len(subset)
        success_rate = ipo_rate + acq_rate
        
        print(f"\n{label}:")
        print(f"N startups:{len(subset):>8,}")
        print(f"IPO rate:{ipo_rate:>7.2f}%")
        print(f"Acquisition rate:{acq_rate:>7.2f}%")
        print(f"Success rate (IPO+M&A):{success_rate:>7.2f}%")

# Calculate difference
hot_subset = finale_usa[finale_usa['hot_ipo_market'] == 1]
cold_subset = finale_usa[finale_usa['hot_ipo_market'] == 0]

ipo_diff = 100 * ((hot_subset['status'] == 'ipo').mean() - (cold_subset['status'] == 'ipo').mean())

print(f"\nINSIGHT:")
print(f"  IPO rate difference: {ipo_diff:+.2f}% (hot vs cold)")

# Exit rates by market cycle
print("\nEXIT RATES BY MARKET CYCLE")

for cycle in ['boom', 'normal', 'bust', 'contraction']:
    subset = finale_usa[finale_usa['market_cycle'] == cycle]
    if len(subset) > 0:
        ipo_rate = 100 * (subset['status'] == 'ipo').sum() / len(subset)
        acq_rate = 100 * (subset['status'] == 'acquired').sum() / len(subset)
        print(f"{cycle:15s} N={len(subset):>7,}  IPO: {ipo_rate:>5.2f}%  M&A: {acq_rate:>5.2f}%")

# =============================================================================
# SAVE FINAL DATASET
# =============================================================================

# Create directory if doesn't exist
config.PROCESSED_PATH.mkdir(parents=True, exist_ok=True)

# Save main dataset
output_path = os.path.join(config.PROCESSED_PATH, 'finale_usa_real_funding.csv')
finale_usa.to_csv(output_path, index=False)

print(f"Dataset saved: {output_path}")
print(f"Shape: {finale_usa.shape}")
print(f"Size: {os.path.getsize(output_path) / (1024**2):.1f} MB")

# Update finale for next steps
finale = finale_usa.copy()

print(f"\nSummary:")
print(f"Startups: {len(finale):,} (USA only)")
print(f"Real funding data: 100%")
print(f"Market conditions: 100%")
print(f"Years: 1975-2013")

---
## 3. Exploratory Data Analysis

Comprehensive analysis of the integrated dataset.

### 3.1 Status Distribution

In [ ]:
# =============================================================================
# STATUS DISTRIBUTION 
# =============================================================================

print("STATUS DISTRIBUTION")

status_counts = finale_usa['status'].value_counts()
status_pct = finale_usa['status'].value_counts(normalize=True) * 100

for status in status_counts.index:
    print(f"{status:15s} {status_counts[status]:>10,} ({status_pct[status]:>6.2f}%)")
print(f"{'TOTAL':15s} {status_counts.sum():>10,}")

# =============================================================================
# VISUALIZATION - 3 PANELS
# =============================================================================

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# ---------------------------------------------------------------------------
# PANEL 1: Bar plot (all statuses)
# ---------------------------------------------------------------------------

status_counts_sorted = status_counts.reindex(['operating', 'acquired', 'closed', 'ipo'])
colors_bar = ['#3498db', '#2ecc71', '#e74c3c', '#f39c12']

status_counts_sorted.plot(kind='bar', ax=axes[0], color=colors_bar, alpha=0.8, width=0.7)
axes[0].set_xlabel('Status', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Count', fontsize=12, fontweight='bold')
axes[0].set_title('Status Distribution (All Startups)', fontsize=14, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3, linestyle='--')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)

# ---------------------------------------------------------------------------
# PANEL 2: Pie chart with EXPLODE + LEGEND
# ---------------------------------------------------------------------------

# Explode small slices for visibility
explode = (0, 0.1, 0.1, 0.15)  # Explode ipo, closed, acquired

wedges, texts, autotexts = axes[1].pie(
    status_counts_sorted, 
    explode=explode,
    autopct='%1.1f%%',
    colors=colors_bar, 
    startangle=90,
    pctdistance=0.85,
    textprops={'fontsize': 11, 'fontweight': 'bold'}
)

# Make percentage text white and bold
for autotext in autotexts:
    autotext.set_color('black')
    autotext.set_fontweight('bold')
    autotext.set_fontsize(11)

# Add legend outside
axes[1].legend(
    status_counts_sorted.index, 
    title="Status",
    loc="center left",
    bbox_to_anchor=(1, 0, 0.5, 1),
    fontsize=11
)

axes[1].set_title('Status Distribution (%)', fontsize=14, fontweight='bold')

# ---------------------------------------------------------------------------
# PANEL 3: Zoomed bar chart (EXITS ONLY - without operating)
# ---------------------------------------------------------------------------

# Focus on exits (exclude operating)
exits_only = status_counts.drop('operating', errors='ignore')
exits_sorted = exits_only.reindex(['acquired', 'closed', 'ipo'])
colors_exits = ['#2ecc71', '#e74c3c', '#f39c12']

exits_sorted.plot(kind='bar', ax=axes[2], color=colors_exits, alpha=0.8, width=0.7)
axes[2].set_xlabel('Exit Type', fontsize=12, fontweight='bold')
axes[2].set_ylabel('Count', fontsize=12, fontweight='bold')
axes[2].set_title('Exit Distribution (Zoom)', fontsize=14, fontweight='bold')
axes[2].grid(axis='y', alpha=0.3, linestyle='--')
axes[2].set_xticklabels(axes[2].get_xticklabels(), rotation=0)

plt.tight_layout()
plt.show()

In [ ]:
# ---------------------------------------------------------------------------
# Helper Function: Cramer's V
# ---------------------------------------------------------------------------

from scipy.stats import chi2_contingency

def cramers_v(x, y):
    """
    Calculate Cramer's V statistic for categorical-categorical association.
    
    V ranges from 0 (no association) to 1 (perfect association)
    Interpretation:
    - 0.00-0.10: Negligible
    - 0.10-0.20: Weak
    - 0.20-0.30: Moderate
    - 0.30+: Strong
    """
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    min_dim = min(confusion_matrix.shape) - 1
    
    if min_dim == 0:
        return 0.0
    
    return np.sqrt(chi2 / (n * min_dim))

# ---------------------------------------------------------------------------
# 1. CREATE FUNDING STAGE CATEGORIES (for this analysis)
# ---------------------------------------------------------------------------

# Categorize funding_rounds into stages
finale_usa['funding_stage'] = pd.cut(
    finale_usa['funding_rounds'],
    bins=[-1, 0, 1, 2, 4, 100],
    labels=['Unfunded', 'Seed (1)', 'Early (2)', 'Growth (3-4)', 'Late (5+)']
)

print("\nFunding Stage Distribution:")
for stage, count in finale_usa['funding_stage'].value_counts().sort_index().items():
    pct = 100 * count / len(finale_usa)
    bar = "█" * int(pct / 2)
    print(f"  {str(stage):15s} {count:>6,} ({pct:>5.1f}%) {bar}")

# ---------------------------------------------------------------------------
# 2. CONTINGENCY TABLE: Funding Stage × Status
# ---------------------------------------------------------------------------
# Absolute counts
contingency = pd.crosstab(
    finale_usa['funding_stage'],
    finale_usa['status'],
    margins=True,
    margins_name='Total'
)

print("ABSOLUTE COUNTS:")
print(contingency)

# Row percentages (most useful)
print("ROW PERCENTAGES (% within each funding stage):")

contingency_pct = pd.crosstab(
    finale_usa['funding_stage'],
    finale_usa['status'],
    normalize='index'
) * 100

print(contingency_pct.round(2))

# ---------------------------------------------------------------------------
# 3. CRAMER'S V: Funding Stage × Status
# ---------------------------------------------------------------------------
# Calculate Cramer's V
v_funding_status = cramers_v(
    finale_usa['funding_stage'].dropna(), 
    finale_usa['status']
)

print(f"Funding Stage x Status")
print(f"Cramer's V = {v_funding_status:.4f}")

# Interpretation
if v_funding_status < 0.1:
    strength = "Negligible"
elif v_funding_status < 0.2:
    strength = "Weak"
elif v_funding_status < 0.3:
    strength = "Moderate"
else:
    strength = "Strong"

print(f"{strength} association")

# Chi-square test
chi2, p_value, dof, expected = chi2_contingency(
    pd.crosstab(finale_usa['funding_stage'].dropna(), finale_usa['status'])
)

print(f"\nChi-square test:")
print(f"χ² = {chi2:.2f}")
print(f"df = {dof}")
print(f"p-value = {p_value:.4e}")

if p_value < 0.001:
    print(f"Highly significant (p < 0.001)")
elif p_value < 0.01:
    print(f"Very significant (p < 0.01)")
elif p_value < 0.05:
    print(f"Significant (p < 0.05)")
else:
    print(f"Not significant (p ≥ 0.05)")

# ---------------------------------------------------------------------------
# 4. ADDITIONAL CRAMER'S V CALCULATIONS
# ---------------------------------------------------------------------------
# Market cycle × Status
if 'market_cycle' in finale_usa.columns:
    v_market_status = cramers_v(
        finale_usa['market_cycle'].dropna(),
        finale_usa['status']
    )
    print(f"Market Cycle x Status:V = {v_market_status:.4f}")

# Category × Status (top 20 categories only - to avoid too many sparse cells)
top_20_categories = finale_usa['category_code'].value_counts().head(20).index
finale_usa_top20 = finale_usa[finale_usa['category_code'].isin(top_20_categories)]

v_sector_status = cramers_v(
    finale_usa_top20['category_code'],
    finale_usa_top20['status']
)
print(f"Sector x Status (top 20):V = {v_sector_status:.4f}")

# Calculate IPO rates and counts
stage_stats = []
for stage in ['Unfunded', 'Seed (1)', 'Early (2)', 'Growth (3-4)', 'Late (5+)']:
    subset = finale_usa[finale_usa['funding_stage'] == stage]
    if len(subset) > 0:
        ipo_rate = 100 * (subset['status'] == 'ipo').mean()
        count = len(subset)
        stage_stats.append({'stage': stage, 'ipo_rate': ipo_rate, 'count': count})

stage_df = pd.DataFrame(stage_stats)
x = range(len(stage_df))

# ---------------------------------------------------------------------------
# 6. KEY INSIGHTS SUMMARY
# ---------------------------------------------------------------------------
# Find stage with highest IPO rate
best_stage = stage_df.loc[stage_df['ipo_rate'].idxmax()]
worst_stage = stage_df.loc[stage_df['ipo_rate'].idxmin()]

print(f"Highest IPO rate:{best_stage['stage']:15s} {best_stage['ipo_rate']:.2f}%")
print(f"Lowest IPO rate:{worst_stage['stage']:15s} {worst_stage['ipo_rate']:.2f}%")

# Overall association strength
print(f"\nAssociation Strengths:")
print(f"Funding Stage - Status:{v_funding_status:.4f} ({strength})")
print(f"Sector - Status:{v_sector_status:.4f}")
if 'market_cycle' in finale_usa.columns:
    print(f"Market Cycle - Status:{v_market_status:.4f}")

In [ ]:
# ---------------------------------------------------------------------------
# 1. Create Angel-Backed Binary Variable
# ---------------------------------------------------------------------------
# Binary: has angel funding or not
finale_usa['angel_backed'] = (finale_usa['angel'] > 0).astype(int)

print("\n✓ Angel-backed Distribution:")
for val, label in [(0, 'No angel funding'), (1, 'Angel-backed')]:
    count = (finale_usa['angel_backed'] == val).sum()
    pct = 100 * count / len(finale_usa)
    bar = "█" * int(pct / 2)
    print(f"{label:20s} {count:>6,} ({pct:>5.1f}%) {bar}")

# ---------------------------------------------------------------------------
# 2. Create Angel Funding Amount Categories
# ---------------------------------------------------------------------------
# # For those with angel funding, categorize by amount
finale_usa['angel_category'] = pd.cut(
    finale_usa['angel'],
    bins=[-1, 0, 250000, 1000000, 100000000],
    labels=['No angel', 'Low (<$250k)', 'Medium ($250k-$1M)', 'High (>$1M)']
)

print("\n✓ Angel Funding Categories:")
for cat, count in finale_usa['angel_category'].value_counts().sort_index().items():
    pct = 100 * count / len(finale_usa)
    bar = "█" * int(pct / 2)
    print(f"{str(cat):25s} {count:>6,} ({pct:>5.1f}%) {bar}")

# ---------------------------------------------------------------------------
# 3. Angel Funding by Status
# ---------------------------------------------------------------------------

for status in ['ipo', 'acquired', 'closed', 'operating']:
    subset = finale_usa[finale_usa['status'] == status]
    angel_rate = 100 * (subset['angel_backed'] == 1).mean()
    count = (subset['angel_backed'] == 1).sum()
    print(f"{status:12s} {angel_rate:>5.1f}% angel-backed ({count:>5,} startups)")

for status in ['ipo', 'acquired', 'closed', 'operating']:
    subset = finale_usa[(finale_usa['status'] == status) & (finale_usa['angel'] > 0)]
    if len(subset) > 0:
        mean_angel = subset['angel'].mean()
        median_angel = subset['angel'].median()
        print(f"{status:12s} Mean: ${mean_angel/1e6:>5.1f}M  Median: ${median_angel/1e6:>5.1f}M  (N={len(subset):,})")

# ---------------------------------------------------------------------------
# 4. Contingency Table: Angel-Backed × Status
# ---------------------------------------------------------------------------
# Absolute counts
contingency_angel = pd.crosstab(
    finale_usa['angel_backed'].map({0: 'No angel', 1: 'Angel-backed'}),
    finale_usa['status'],
    margins=True,
    margins_name='Total'
)

print("ABSOLUTE COUNTS:")
print(contingency_angel)

# Row percentages

contingency_angel_pct = pd.crosstab(
    finale_usa['angel_backed'].map({0: 'No angel', 1: 'Angel-backed'}),
    finale_usa['status'],
    normalize='index'
) * 100

print(contingency_angel_pct.round(2))

# ---------------------------------------------------------------------------
# 5. Cramer's V: Angel-Backed × Status
# ---------------------------------------------------------------------------
# Calculate Cramer's V
v_angel_status = cramers_v(finale_usa['angel_backed'], finale_usa['status'])

# Interpretation
if v_angel_status < 0.1:
    strength = "Negligible"
elif v_angel_status < 0.2:
    strength = "Weak"
elif v_angel_status < 0.3:
    strength = "Moderate"
else:
    strength = "Strong"

print(f"{strength} association")

# Chi-square test
chi2, p_value, dof, expected = chi2_contingency(
    pd.crosstab(finale_usa['angel_backed'], finale_usa['status'])
)

print(f"\nChi-square test:")
print(f"χ² = {chi2:.2f}")
print(f"df = {dof}")
print(f"p-value = {p_value:.4e}")

if p_value < 0.001:
    print(f"Highly significant (p < 0.001)")
elif p_value < 0.01:
    print(f"Very significant (p < 0.01)")
elif p_value < 0.05:
    print(f"Significant (p < 0.05)")
else:
    print(f"Not significant (p ≥ 0.05)")

v_angel_amount = cramers_v(finale_usa['angel_category'].dropna(), finale_usa['status'])

print(f"Cramer's V = {v_angel_amount:.4f}")

# Calculate IPO rates
angel_ipo = []
for val, label in [(0, 'No angel'), (1, 'Angel-backed')]:
    subset = finale_usa[finale_usa['angel_backed'] == val]
    ipo_rate = 100 * (subset['status'] == 'ipo').mean()
    count = len(subset)
    angel_ipo.append({'group': label, 'ipo_rate': ipo_rate, 'count': count})

angel_ipo_df = pd.DataFrame(angel_ipo)
x = range(len(angel_ipo_df))

# Calculate exit rates by angel category
angel_exits = []
for cat in ['No angel', 'Low (<$250k)', 'Medium ($250k-$1M)', 'High (>$1M)']:
    subset = finale_usa[finale_usa['angel_category'] == cat]
    if len(subset) > 0:
        ipo_rate = 100 * (subset['status'] == 'ipo').mean()
        ma_rate = 100 * (subset['status'] == 'acquired').mean()
        angel_exits.append({'category': cat, 'ipo': ipo_rate, 'ma': ma_rate})

exits_df = pd.DataFrame(angel_exits)
x = range(len(exits_df))

# Filter only angel-backed startups
angel_funded = finale_usa[finale_usa['angel'] > 0].copy()

# ---------------------------------------------------------------------------
# 6. KEY INSIGHTS
# ---------------------------------------------------------------------------
# Angel-backed rate by status
ipo_angel_rate = 100 * (finale_usa[finale_usa['status']=='ipo']['angel_backed']==1).mean()
noipo_angel_rate = 100 * (finale_usa[finale_usa['status']!='ipo']['angel_backed']==1).mean()

print(f"Angel-Backed Rates:")
print(f"IPO companies:{ipo_angel_rate:.1f}% angel-backed")
print(f"Non-IPO companies:{noipo_angel_rate:.1f}% angel-backed")
print(f"Difference:{ipo_angel_rate - noipo_angel_rate:+.1f}%")

# IPO rate comparison
ipo_rate_angel = 100 * (finale_usa[finale_usa['angel_backed']==1]['status']=='ipo').mean()
ipo_rate_no_angel = 100 * (finale_usa[finale_usa['angel_backed']==0]['status']=='ipo').mean()

print(f"\nIPO Rates:")
print(f"Angel-backed:{ipo_rate_angel:.2f}%")
print(f"Not angel-backed:{ipo_rate_no_angel:.2f}%")
print(f"Ratio:{ipo_rate_angel/ipo_rate_no_angel:.2f}x")

# Amount effect
if len(finale_usa[(finale_usa['angel_category']=='High (>$1M)') & (finale_usa['status']=='ipo')]) > 0:
    high_angel_ipo = 100 * (finale_usa[finale_usa['angel_category']=='High (>$1M)']['status']=='ipo').mean()
    low_angel_ipo = 100 * (finale_usa[finale_usa['angel_category']=='Low (<$250k)']['status']=='ipo').mean()
    
    print(f"\nAngel Amount Impact:")
    print(f"High angel (>$1M):{high_angel_ipo:.2f}% IPO rate")
    print(f"Low angel (<$250k):{low_angel_ipo:.2f}% IPO rate")

print(f"\nAssociation Strength:")
print(f"Angel-backed x Status:{v_angel_status:.4f} ({strength})")
print(f"Angel amount x Status:{v_angel_amount:.4f}")

In [ ]:
for sector in finale_usa['category_code'].cat.categories:
    subset = finale_usa[finale_usa['category_code'] == sector]
    
    # Angel-backed
    angel_subset = subset[subset['angel_backed'] == 1]
    angel_ipo_rate = 100 * (angel_subset['status'] == 'ipo').mean() if len(angel_subset) > 0 else 0
    angel_n = len(angel_subset)
    
    # Non-angel
    no_angel_subset = subset[subset['angel_backed'] == 0]
    no_angel_ipo_rate = 100 * (no_angel_subset['status'] == 'ipo').mean() if len(no_angel_subset) > 0 else 0
    no_angel_n = len(no_angel_subset)
    
    # Only print if both groups exist
    if angel_n > 10 and no_angel_n > 10:  # Min 10 startup per gruppo
        diff = angel_ipo_rate - no_angel_ipo_rate
        print(f"{sector:20s} Angel: {angel_ipo_rate:>5.2f}% (N={angel_n:>4,})  "
              f"No-angel: {no_angel_ipo_rate:>5.2f}% (N={no_angel_n:>4,})  "
              f"Diff: {diff:>+6.2f}%")

In [ ]:
# Check if angel + VC changes the pattern
finale_usa['has_multiple_rounds'] = (finale_usa['funding_rounds'] >= 3).astype(int)

# Angel + Multiple rounds vs Angel alone
angel_multi = finale_usa[(finale_usa['angel_backed']==1) & 
                         (finale_usa['has_multiple_rounds']==1)]
angel_solo = finale_usa[(finale_usa['angel_backed']==1) & 
                        (finale_usa['has_multiple_rounds']==0)]

print(f"Angel + VC: {(angel_multi['status']=='ipo').mean():.2%} IPO rate")
print(f"Angel only: {(angel_solo['status']=='ipo').mean():.2%} IPO rate")

In [ ]:
# ---------------------------------------------------------------------------
# 1. Basic Relationship: angel vs funding_total_usd
# ---------------------------------------------------------------------------

# Check if angel is a SUBSET of funding_total_usd
subset_check = (finale_usa['angel'] <= finale_usa['funding_total_usd']).all()

print(f"\nIs angel ≤ funding_total_usd for ALL rows? {subset_check}")

if subset_check:
    print("angel appears to be a SUBSET of total funding")
else:
    # Find violations
    violations = finale_usa[finale_usa['angel'] > finale_usa['funding_total_usd']]
    print(f"Found {len(violations)} cases where angel > total funding (data error?)")

# ---------------------------------------------------------------------------
# 2. Proportion Analysis
# ---------------------------------------------------------------------------
# For angel-backed startups, what % of total funding is angel?
angel_backed = finale_usa[finale_usa['angel'] > 0].copy()

if len(angel_backed) > 0:
    angel_backed['angel_pct'] = 100 * angel_backed['angel'] / angel_backed['funding_total_usd']
    
    print(f"\nFor angel-backed startups (N={len(angel_backed):,}):")
    print(f"Angel as % of total funding:")
    print(f"Mean:{angel_backed['angel_pct'].mean():.1f}%")
    print(f"Median:{angel_backed['angel_pct'].median():.1f}%")
    print(f"Q1:{angel_backed['angel_pct'].quantile(0.25):.1f}%")
    print(f"Q3:{angel_backed['angel_pct'].quantile(0.75):.1f}%")
    
    # Cases where angel = 100% of funding (angel-only)
    angel_only = (angel_backed['angel_pct'] > 99).sum()
    print(f"\nAngel-only (100% of funding):{angel_only:,} ({100*angel_only/len(angel_backed):.1f}%)")
    
    # Cases where angel < 50% (has other funding)
    angel_plus = (angel_backed['angel_pct'] < 50).sum()
    print(f"Angel + Other (angel <50%):{angel_plus:,} ({100*angel_plus/len(angel_backed):.1f}%)")

# ---------------------------------------------------------------------------
# 3. Check for VC-related columns
# ---------------------------------------------------------------------------
# Search for columns with 'vc', 'series', 'round' in name
vc_related = [col for col in finale_usa.columns if any(x in col.lower() for x in ['vc', 'series', 'round', 'seed'])]

print(f"\nFound{len(vc_related)} potentially VC-related columns:")
for col in vc_related:
    dtype = finale_usa[col].dtype
    nunique = finale_usa[col].nunique()
    non_null = finale_usa[col].notna().sum()
    print(f"  • {col:30s} {str(dtype):15s} {nunique:>6} unique ({non_null:>7,} non-null)")

# ---------------------------------------------------------------------------
# 4. Compare angel vs funding characteristics
# ---------------------------------------------------------------------------
print(f"\nAngel funding:")
print(f"Non-zero:{(finale_usa['angel'] > 0).sum():,} ({100*(finale_usa['angel'] > 0).mean():.1f}%)")
print(f"Median (if >0):${finale_usa[finale_usa['angel'] > 0]['angel'].median()/1e6:.1f}M")
print(f"Mean (if >0):${finale_usa[finale_usa['angel'] > 0]['angel'].mean()/1e6:.1f}M")

print(f"\nTotal funding:")
print(f"Non-zero:{(finale_usa['funding_total_usd'] > 0).sum():,} ({100*(finale_usa['funding_total_usd'] > 0).mean():.1f}%)")
print(f"Median (if >0):${finale_usa[finale_usa['funding_total_usd'] > 0]['funding_total_usd'].median()/1e6:.1f}M")
print(f"Mean (if >0):${finale_usa[finale_usa['funding_total_usd'] > 0]['funding_total_usd'].mean()/1e6:.1f}M")

# ---------------------------------------------------------------------------
# 5. Outlier Analysis
# ---------------------------------------------------------------------------
# Top 10 angel amounts
print("\nTop 10 angel funding amounts:")
top_angel = finale_usa.nlargest(10, 'angel')[['angel', 'funding_total_usd', 'funding_rounds', 'status']]

for idx, row in top_angel.iterrows():
    angel_amt = row['angel'] / 1e6
    total_amt = row['funding_total_usd'] / 1e6
    rounds = row['funding_rounds']
    status = row['status']
    pct = 100 * row['angel'] / row['funding_total_usd'] if row['funding_total_usd'] > 0 else 0
    
    print(f"  Angel: ${angel_amt:>6.1f}M  Total: ${total_amt:>6.1f}M  "
          f"({pct:>5.1f}%)  Rounds: {rounds:>2.0f}  Status: {status}")

# ---------------------------------------------------------------------------
# 6. Sample Cases
# ---------------------------------------------------------------------------
# Case 1: High angel, low total (angel-only?)
high_angel_low_total = finale_usa[
    (finale_usa['angel'] > 1e6) & 
    (finale_usa['angel'] / finale_usa['funding_total_usd'] > 0.9)
].head(3)

print("\nCase 1: High angel amount, mostly angel (>90% of total):")
for idx, row in high_angel_low_total.iterrows():
    print(f"  Angel: ${row['angel']/1e6:.1f}M  Total: ${row['funding_total_usd']/1e6:.1f}M  "
          f"Rounds: {row['funding_rounds']:.0f}  Status: {row['status']}")

# Case 2: High angel, high total (angel + VC?)
high_angel_high_total = finale_usa[
    (finale_usa['angel'] > 500000) & 
    (finale_usa['funding_total_usd'] > 10e6) &
    (finale_usa['angel'] / finale_usa['funding_total_usd'] < 0.1)
].head(3)

print("\nCase 2: High angel, but small % of total (angel + big VC?):")
for idx, row in high_angel_high_total.iterrows():
    pct = 100 * row['angel'] / row['funding_total_usd']
    print(f"  Angel: ${row['angel']/1e6:.1f}M ({pct:.1f}%)  Total: ${row['funding_total_usd']/1e6:.1f}M  "
          f"Rounds: {row['funding_rounds']:.0f}  Status: {row['status']}")

In [ ]:
# =============================================================================
# NETWORK EFFECTS ANALYSIS (Alternative to Funding Sources)
# =============================================================================
print("NETWORK EFFECTS ANALYSIS")

# Create network categories
finale_usa['has_person_financed'] = (finale_usa['person_financed'] > 0).astype(int)
finale_usa['has_startup_financed'] = (finale_usa['startup_financed'] > 0).astype(int)
finale_usa['has_fin_org_financed'] = (finale_usa['fin_org_financed'] > 0).astype(int)

# Cramer's V for network variables
v_person = cramers_v(finale_usa['has_person_financed'], finale_usa['status'])
v_startup = cramers_v(finale_usa['has_startup_financed'], finale_usa['status'])
v_org = cramers_v(finale_usa['has_fin_org_financed'], finale_usa['status'])

print(f"Person financed x Status:V = {v_person:.4f}")
print(f"Startup financed x Status:V = {v_startup:.4f}")
print(f"Fin org financed x Status:V = {v_org:.4f}")

# Mosaic plot equivalent
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for idx, (var, title) in enumerate([
    ('has_person_financed', 'Person Financed'),
    ('has_startup_financed', 'Startup Financed'),
    ('has_fin_org_financed', 'Fin Org Financed')
]):
    
    ct = pd.crosstab(finale_usa[var], finale_usa['status'], normalize='index')
    ct.plot(kind='bar', stacked=True, ax=axes[idx],
            color=['#f39c12', '#2ecc71', '#e74c3c', '#3498db'])
    
    axes[idx].set_title(f'{title} x Status', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(f'{title} (0=No, 1=Yes)', fontsize=11, fontweight='bold')
    axes[idx].set_ylabel('Proportion', fontsize=11, fontweight='bold')
    axes[idx].set_xticklabels(['No', 'Yes'], rotation=0)
    axes[idx].legend(title='Status', bbox_to_anchor=(1.05, 1))
    axes[idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Create investment intensity categories
finale_usa['investment_intensity'] = pd.cut(
    finale_usa['investment_rounds'],
    bins=[-1, 0, 2, 5, 100],
    labels=['No investment', 'Low (1-2)', 'Medium (3-5)', 'High (6+)']
)

# Cramer's V
v_investment = cramers_v(finale_usa['investment_intensity'].dropna(), finale_usa['status'])
print(f"Investment intensity x Status: V = {v_investment:.4f}")

# Mosaic plot
ct = pd.crosstab(finale_usa['investment_intensity'], finale_usa['status'], normalize='index')
ct.plot(kind='bar', stacked=True, figsize=(16, 6))

In [ ]:
# Status by sector
sector_data = finale_usa[
    finale_usa['category_code'].notna() & (finale_usa['category_code'] != '')
].copy()

status_settore = sector_data.groupby('category_code').agg({
    'id': 'count',
    'milestones': 'mean'
}).rename(columns={'id': 'n'})

# Add status counts
for status_val in ['acquired', 'closed', 'ipo', 'operating']:
    status_settore[f'n_{status_val}'] = sector_data[
        sector_data['status'] == status_val
    ].groupby('category_code').size()

status_settore = status_settore.fillna(0)

# Calculate rates
for status_val in ['acquired', 'closed', 'ipo', 'operating']:
    status_settore[f'{status_val}_rate'] = status_settore[f'n_{status_val}'] / status_settore['n']

status_settore = status_settore.reset_index()
status_settore = status_settore.sort_values('n', ascending=False)

print("\nTop 20 sectors by number of startups:")
print(status_settore.head(20)[['category_code', 'n', 'ipo_rate', 'acquired_rate']])

In [ ]:
# Four separate plots for IPO, acquired, closed, operating rates
fig, axes = plt.subplots(4, 1, figsize=(12, 28))

# Sort by different rates for each plot
status_settore_ipo = status_settore.sort_values('ipo_rate', ascending=True)
status_settore_acq = status_settore.sort_values('acquired_rate', ascending=True)
status_settore_closed = status_settore.sort_values('closed_rate', ascending=True)
status_settore_op = status_settore.sort_values('operating_rate', ascending=True)

# IPO Rate
axes[0].barh(status_settore_ipo['category_code'], 
             status_settore_ipo['ipo_rate']*100, 
             color='gold', alpha=0.7)
axes[0].set_xlabel('Percentage of listed companies (%)')
axes[0].set_title('Percentage of Listed Companies by Sector', fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# Acquired Rate
axes[1].barh(status_settore_acq['category_code'], 
             status_settore_acq['acquired_rate']*100,
             color='green', alpha=0.7)
axes[1].set_xlabel('Percentage of acquired companies (%)')
axes[1].set_title('Percentage of Acquired Companies by Sector', fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)

# Closed Rate
axes[2].barh(status_settore_closed['category_code'], 
             status_settore_closed['closed_rate']*100,
             color='red', alpha=0.7)
axes[2].set_xlabel('Percentage of closed companies (%)')
axes[2].set_title('Percentage of Closed Companies by Sector', fontweight='bold')
axes[2].grid(axis='x', alpha=0.3)

# Operating Rate
axes[3].barh(status_settore_op['category_code'], 
             status_settore_op['operating_rate']*100,
             color='blue', alpha=0.7)
axes[3].set_xlabel('Percentage of operating companies (%)')
axes[3].set_title('Percentage of Operating Companies by Sector', fontweight='bold')
axes[3].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

### 3.3 Geographic Analysis

In [ ]:
# Use USA dataset, group by state
status_state = finale_usa[
    finale_usa['state_code'].notna() & (finale_usa['state_code'] != '')
].groupby('state_code').agg({
    'id': 'count'
}).rename(columns={'id': 'n'})

# Add status counts
for status_val in ['ipo', 'acquired', 'closed', 'operating']:
    status_state[f'n_{status_val}'] = finale_usa[
        finale_usa['status'] == status_val
    ].groupby('state_code').size()

status_state = status_state.fillna(0)

# Calculate rates
for status_val in ['ipo', 'acquired', 'closed', 'operating']:
    status_state[f'{status_val}_rate'] = (status_state[f'n_{status_val}'] / status_state['n']) * 100

status_state = status_state.reset_index().sort_values('n', ascending=False)

# Print top 20 states
print("TOP 20 STATES")
print(f"{'State':<8} {'N Startups':>12} {'IPO %':>8} {'M&A %':>8} {'Closed %':>8}")


for idx, row in status_state.head(20).iterrows():
    print(f"{row['state_code']:<8} {int(row['n']):>12,} {row['ipo_rate']:>7.2f}% {row['acquired_rate']:>7.2f}% {row['closed_rate']:>7.2f}%")

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# Panel 1: Top 20 states by count
top_20_states = status_state.nlargest(20, 'n').sort_values('n')

axes[0,0].barh(top_20_states['state_code'], top_20_states['n'], color='blue', alpha=0.7)
axes[0,0].set_xlabel('Number of Startups', fontsize=12, fontweight='bold')
axes[0,0].set_ylabel('State', fontsize=12, fontweight='bold')
axes[0,0].set_title('Top 20 US States by Number of Startups', fontsize=14, fontweight='bold')
axes[0,0].grid(axis='x', alpha=0.3)

# Add value labels
for i, (idx, row) in enumerate(top_20_states.iterrows()):
    axes[0,0].text(row['n'] + 100, i, f"{int(row['n']):,}", va='center', fontsize=9)

# Panel 2: Top 10 states by IPO rate (min 50 startups)
top_ipo_rate = status_state[status_state['n'] >= 50].nlargest(10, 'ipo_rate').sort_values('ipo_rate')

axes[0,1].barh(top_ipo_rate['state_code'], top_ipo_rate['ipo_rate'], color='green', alpha=0.7)
axes[0,1].set_xlabel('IPO Rate (%)', fontsize=12, fontweight='bold')
axes[0,1].set_ylabel('State', fontsize=12, fontweight='bold')
axes[0,1].set_title('Top 10 States by IPO Rate (min 50 startups)', fontsize=14, fontweight='bold')
axes[0,1].grid(axis='x', alpha=0.3)

# Add value labels
for i, (idx, row) in enumerate(top_ipo_rate.iterrows()):
    axes[0,1].text(row['ipo_rate'] + 0.1, i, f"{row['ipo_rate']:.1f}%", va='center', fontsize=9)

# Panel 3: Top 10 states by M&A rate (min 50 startups)
top_acq_rate = status_state[status_state['n'] >= 50].nlargest(10, 'acquired_rate').sort_values('acquired_rate')

axes[1,0].barh(top_acq_rate['state_code'], top_acq_rate['acquired_rate'], color='yellow', alpha=0.7)
axes[1,0].set_xlabel('M&A Rate (%)', fontsize=12, fontweight='bold')
axes[1,0].set_ylabel('State', fontsize=12, fontweight='bold')
axes[1,0].set_title('Top 10 States by M&A Rate (min 50 startups)', fontsize=14, fontweight='bold')
axes[1,0].grid(axis='x', alpha=0.3)

# Add value labels
for i, (idx, row) in enumerate(top_acq_rate.iterrows()):
    axes[1,0].text(row['acquired_rate'] + 0.2, i, f"{row['acquired_rate']:.1f}%", va='center', fontsize=9)

# Panel 4: Top 10 states by Success rate (IPO + M&A, min 50 startups)
status_state['success_rate'] = status_state['ipo_rate'] + status_state['acquired_rate']
top_success = status_state[status_state['n'] >= 50].nlargest(10, 'success_rate').sort_values('success_rate')

axes[1,1].barh(top_success['state_code'], top_success['success_rate'], color='purple', alpha=0.7)
axes[1,1].set_xlabel('Success Rate (IPO + M&A %)', fontsize=12, fontweight='bold')
axes[1,1].set_ylabel('State', fontsize=12, fontweight='bold')
axes[1,1].set_title('Top 10 States by Success Rate (min 50 startups)', fontsize=14, fontweight='bold')
axes[1,1].grid(axis='x', alpha=0.3)

# Add value labels
for i, (idx, row) in enumerate(top_success.iterrows()):
    axes[1,1].text(row['success_rate'] + 0.3, i, f"{row['success_rate']:.1f}%", va='center', fontsize=9)

plt.tight_layout()
plt.show()

print("\nUSA state analysis complete")

In [ ]:
# =============================================================================
# ENHANCED HEATMAP - PROFESSIONAL VERSION
# =============================================================================

import folium
from folium.plugins import HeatMap, MiniMap, Fullscreen, MarkerCluster
import json

# Load offices data
offices_path = os.path.join(config.RAW_DIR, "offices.csv")

if offices is not None:
    # Merge finale_usa with offices to get coordinates
    finale_with_geo = finale_usa.merge(
        offices[['object_id', 'latitude', 'longitude', 'city', 'state_code']],
        left_on='id',
        right_on='object_id',
        how='left'
    )
    
    # Filter only startups with valid coordinates
    geo_data = finale_with_geo[
        finale_with_geo['latitude'].notna() & 
        finale_with_geo['longitude'].notna() &
        (finale_with_geo['latitude'].between(24, 50)) &  # USA bounds
        (finale_with_geo['longitude'].between(-125, -65))
    ].copy()
    
    print(f"Startups with coordinates: {len(geo_data):,} ({100*len(geo_data)/len(finale_usa):.1f}%)")
    
    # =========================================================================
    # CREATE BASE MAP
    # =========================================================================
    
    m = folium.Map(
        location=[39.8283, -98.5795],
        zoom_start=4,
        tiles='CartoDB dark_matter',
        control_scale=True
    )
    
    # =========================================================================
    # LAYER 1: HEATMAP
    # =========================================================================
    
    # Create heatmap data with WEIGHTED by status
    heat_data_weighted = []
    
    for idx, row in geo_data.iterrows():
        # Weight by importance: IPO = 5x, Acquired = 3x, Others = 1x
        if row['status'] == 'ipo':
            weight = 5
        elif row['status'] == 'acquired':
            weight = 3
        else:
            weight = 1
        
        heat_data_weighted.append([row['latitude'], row['longitude'], weight])
    
    # Main heatmap layer
    heatmap_layer = folium.FeatureGroup(name='Density Heatmap', show=True)
    
    HeatMap(
        heat_data_weighted,
        min_opacity=0.3,
        max_zoom=13,
        radius=15,
        blur=20,
        gradient={0.4: 'blue', 0.65: 'lime', 0.8: 'yellow', 1.0: 'red'}
    ).add_to(heatmap_layer)
    
    heatmap_layer.add_to(m)
    
    # =========================================================================
    # LAYER 2: TOP CITIES MARKERS 
    # =========================================================================
    
    cities_layer = folium.FeatureGroup(name='Top Cities', show=False)
    
    # Aggregate by city
    city_stats = geo_data.groupby('city').agg({
        'id': 'count',
        'latitude': 'first',
        'longitude': 'first',
        'status': lambda x: (x.isin(['ipo', 'acquired'])).sum()
    }).rename(columns={'id': 'total', 'status': 'success'})
    
    city_stats['success_rate'] = (city_stats['success'] / city_stats['total']) * 100
    city_stats = city_stats.reset_index()
    
    # Top 20 cities
    top_cities = city_stats.nlargest(20, 'total')
    
    for idx, city in top_cities.iterrows():
        # Size by number of startups
        radius = min(5 + (city['total'] / 100), 30)
        
        # Color by success rate
        if city['success_rate'] > 15:
            color = '#2ecc71'  # Green
        elif city['success_rate'] > 10:
            color = '#f39c12'  # Orange
        else:
            color = '#e74c3c'  # Red
        
        folium.CircleMarker(
            location=[city['latitude'], city['longitude']],
            radius=radius,
            popup=f"""
            <div style="font-family: Arial; color: #2c3e50;">
                <h4>{city['city']}</h4>
                <p><b>Total Startups:</b> {int(city['total']):,}<br>
                <b>Successes:</b> {int(city['success'])}<br>
                <b>Success Rate:</b> {city['success_rate']:.1f}%</p>
            </div>
            """,
            tooltip=f"{city['city']}: {int(city['total']):,} startups",
            color=color,
            fill=True,
            fillColor=color,
            fillOpacity=0.7,
            weight=2
        ).add_to(cities_layer)
    
    cities_layer.add_to(m)
    
    # =========================================================================
    # LAYER 3: SUCCESS STORIES (IPO + Acquired)
    # =========================================================================
    
    success_layer = folium.FeatureGroup(name='Success Stories', show=False)
    
    success_cluster = MarkerCluster(
        name='Success Cluster',
        overlay=True,
        control=False
    ).add_to(success_layer)
    
    success_data = geo_data[geo_data['status'].isin(['ipo', 'acquired'])]
    
    for idx, row in success_data.iterrows():
        icon_color = 'red' if row['status'] == 'ipo' else 'green'
        
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=3,
            popup=f"""
            <div style="font-family: Arial;">
                <h4 style="color: {icon_color};">{row['status'].upper()}</h4>
                <p><b>City:</b> {row.get('city', 'N/A')}<br>
                <b>Sector:</b> {row.get('category_code', 'N/A')}<br>
                <b>Funding:</b> ${row.get('funding_total_usd', 0)/1e6:.1f}M</p>
            </div>
            """,
            color=icon_color,
            fill=True,
            fillColor=icon_color,
            fillOpacity=0.8,
            weight=1
        ).add_to(success_cluster)
    
    success_layer.add_to(m)
    
    # =========================================================================
    # ADD PROFESSIONAL FEATURES
    # =========================================================================
    
    # 1. Fullscreen button
    Fullscreen(
        position='topleft',
        title='Fullscreen',
        title_cancel='Exit fullscreen',
        force_separate_button=True
    ).add_to(m)
    
    # 2. MiniMap (overview)
    minimap = MiniMap(
        tile_layer='CartoDB dark_matter',
        position='bottomleft',
        width=150,
        height=150,
        collapsed_width=25,
        collapsed_height=25,
        zoom_level_offset=-5
    )
    m.add_child(minimap)
    
    # 3. Statistics Panel (top right)
    stats_html = f"""
    <div style="position: fixed; 
                top: 10px; right: 10px; 
                width: 300px; 
                background-color: white; 
                border: 2px solid blue;
                border-radius: 10px;
                box-shadow: 0 4px 6px rgba(0,0,0,0.3);
                z-index: 9999; 
                font-family: Arial;
                padding: 15px;
                color: black;">
        
        <h3 style="margin-top: 0; color: blue; border-bottom: 2px solid blue; padding-bottom: 10px;">
            USA Startup Heatmap
        </h3>
        
        <div style="font-size: 13px;">
            <p style="margin: 8px 0;"><b>Total Mapped:</b> {len(geo_data):,} startups</p>
            <p style="margin: 8px 0;"><b>Coverage:</b> {100*len(geo_data)/len(finale_usa):.1f}% of dataset</p>
            <p style="margin: 8px 0;"><b>Period:</b> {int(geo_data['first_funding_year'].min())}-{int(geo_data['first_funding_year'].max())}</p>
        </div>
        
        <h4 style="color: blue; margin-top: 15px; margin-bottom: 10px;">Hotspots</h4>
        
        <h4 style="color: blue; margin-top: 15px; margin-bottom: 10px;">Exit Metrics</h4>
        
        <div style="font-size: 11px;">
            <p style="margin: 5px 0;"><b>IPO:</b> {(geo_data['status']=='ipo').sum():,} ({100*(geo_data['status']=='ipo').mean():.1f}%)</p>
            <p style="margin: 5px 0;"><b>M&A:</b> {(geo_data['status']=='acquired').sum():,} ({100*(geo_data['status']=='acquired').mean():.1f}%)</p>
            <p style="margin: 5px 0;"><b>Success:</b> {100*(geo_data['status'].isin(['ipo','acquired'])).mean():.1f}%</p>
        </div>
    </div>
    """
    
    m.get_root().html.add_child(folium.Element(stats_html))
    
    # 4. Legend (bottom right)
    legend_html = f"""
    <div style="position: fixed; 
                bottom: 30px; right: 10px; 
                width: 250px; 
                background-color: white; 
                border: 2px solid blue;
                border-radius: 10px;
                box-shadow: 0 4px 6px rgba(0,0,0,0.3);
                z-index: 9999; 
                font-family: Arial;
                padding: 12px;
                font-size: 12px;
                color: balck;">
        
        <h4 style="margin-top: 0; color: blue; border-bottom: 1px solid blue; padding-bottom: 8px;">
            Heatmap Legend
        </h4>
        
        <p style="font-size: 11px; color: black; margin: 8px 0;">
            Density weighted by exit success:
        </p>
        
        <div style="margin: 8px 0;">
            <div style="background: linear-gradient(to right, blue, lime, yellow, red); 
                        height: 20px; border-radius: 5px;"></div>
            <div style="display: flex; justify-content: space-between; font-size: 10px; margin-top: 5px;">
                <span>Low</span>
                <span>High</span>
            </div>
        </div>
        
        <div style="margin-top: 12px; font-size: 11px;">
            <p style="margin: 5px 0;">• IPO = 5x weight</p>
            <p style="margin: 5px 0;">• M&A = 3x weight</p>
            <p style="margin: 5px 0;">• Others = 1x weight</p>
        </div>
        
        <p style="margin-top: 10px; padding-top: 8px; border-top: 1px solid rgba(255,255,255,0.2); 
                  font-size: 10px; color: #95a5a6;">
            Zoom in to see individual startups<br>
        </p>
    </div>
    """
    
    m.get_root().html.add_child(folium.Element(legend_html))
    
    # 5. Layer Control
    folium.LayerControl(position='topright', collapsed=False).add_to(m)
    
    # =========================================================================
    # SAVE MAP
    # =========================================================================
    
    heatmap_path = os.path.join(config.DATA_DIR, 'startup_heatmap_enhanced.html')
    m.save(heatmap_path)
    
    print(f"\nENHANCED HEATMAP CREATED")
    print(f"Saved to: {heatmap_path}")
    print(f"File size: {os.path.getsize(heatmap_path) / 1024:.1f} KB")
    
    # Display in notebook
    display(m)

else:
    print("offices.csv not found")